# Date time location

In [1]:
import cv2
from datetime import datetime
import requests
import time

def get_current_location():
    ip_info_url = "http://ipinfo.io/json"
    response = requests.get(ip_info_url)
    data = response.json()
    
    city = data.get('city', 'Unknown City')
    region = data.get('region', 'Unknown Region')
    country = data.get('country', 'Unknown Country')
    
    # Combine location details
    location = f"{city}, {region}, {country}"
    
    return location

# Open video capture
cap = cv2.VideoCapture(0)

# Set resolution and frame rate (optional)
width = 1280
height = 720
fps = 30

cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FPS, fps)

cv2.namedWindow('Live Camera Feed', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Live Camera Feed', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

last_update_time = time.time()
update_interval = 3  # Update every 3 seconds
display_index = 0

location_text = get_current_location()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]

    current_time = datetime.now().strftime('%H:%M:%S')
    current_date = datetime.now().strftime('%d-%m-%Y')

    # Update display content every `update_interval` seconds
    if time.time() - last_update_time >= update_interval:
        display_index = (display_index + 1) % 3  # Cycle through 0, 1, 2
        last_update_time = time.time()

    # Determine what to display
    if display_index == 0:
        text_to_display = current_time
    elif display_index == 1:
        text_to_display = current_date
    else:
        text_to_display = location_text

    # Get the text size to calculate the position at the bottom right
    text_size = cv2.getTextSize(text_to_display, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]

    # Calculate position at the bottom right with added space
    text_x = frame_width - text_size[0] - 10
    text_y = frame_height - 10

    cv2.putText(frame, text_to_display, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('Live Camera Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
from datetime import datetime
import requests
import time

def get_current_location():
    ip_info_url = "http://ipinfo.io/json"
    response = requests.get(ip_info_url)
    data = response.json()
    
    city = data.get('city', 'Unknown City')
    country = data.get('country', 'Unknown Country')
    
    return city, country

# Open video capture
cap = cv2.VideoCapture(0)

# Set resolution and frame rate (optional)
width = 1280
height = 720
fps = 30

cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FPS, fps)

cv2.namedWindow('Live Camera Feed', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Live Camera Feed', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

last_update_time = time.time()
update_interval = 3  # Update every 3 seconds
display_index = 0

city, country = get_current_location()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]

    current_time = datetime.now().strftime('%H:%M:%S')
    current_date = datetime.now().strftime('%d-%m-%Y')
    location_text = f"{city}, {country}"

    # Update display content every `update_interval` seconds
    if time.time() - last_update_time >= update_interval:
        display_index = (display_index + 1) % 3  # Cycle through 0, 1, 2
        last_update_time = time.time()

    # Determine what to display
    if display_index == 0:
        text_to_display = current_time
    elif display_index == 1:
        text_to_display = current_date
    else:
        text_to_display = location_text

    # Get the text size to calculate the position at the bottom right
    text_size = cv2.getTextSize(text_to_display, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]

    # Calculate position at the bottom right with added space
    text_x = frame_width - text_size[0] - 10
    text_y = frame_height - 10

    cv2.putText(frame, text_to_display, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('Live Camera Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Head position

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance
import time
from datetime import datetime
import pyttsx3
import threading

# Initialize Mediapipe components
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh

# Initialize pyttsx3 engine for voice alerts
engine = pyttsx3.init()

def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def calculate_mar(mouth):
    A = distance.euclidean(mouth[2], mouth[6])  # Vertical distance
    B = distance.euclidean(mouth[3], mouth[5])  # Vertical distance
    C = distance.euclidean(mouth[0], mouth[4])  # Horizontal distance
    return (A + B) / (2.0 * C)

# Define constants
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
MOUTH_AR_THRESH = 0.7
MOUTH_AR_CONSEC_FRAMES = 5
YAWN_ALERT_CONSEC_FRAMES = 3
DROWSY_ALERT_SECONDS = 20
WARNING_DISPLAY_SECONDS = 10
HEAD_POSITION_ALERT_SECONDS = 5
COUNT_INTERVAL = 30  # Interval to update and display counts

# Initialize counters and time tracking
blink_count = 0
yawn_count = 0
blink_frames = 0
yawn_frames = 0
consec_yawn_frames = 0
last_yawn_count = 0
increasing_yawn_streak = 0
last_blink_update_time = time.time()
count_update_time = time.time()  # Timer for counting interval

warning_message = ""
warning_start_time = 0
head_position_start_time = None
position = 'center'

# Open video capture with 720p resolution
cap = cv2.VideoCapture(0)

# Set the resolution and frame rate to 720p
width = 1280
height = 720
fps = 30

cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FPS, fps)

cv2.namedWindow('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True) as face_mesh:
    with mp_face_detection.FaceDetection(min_detection_confidence=0.1) as face_detection:

        def speak(message):
            def speak_thread():
                engine.say(message)
                engine.runAndWait()
            
            threading.Thread(target=speak_thread).start()

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            frame_height, frame_width = frame.shape[:2]
            center_x, center_y = frame_width // 2, frame_height // 2
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_detection.process(rgb_frame)

            if results.detections:
                for detection in results.detections:
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = frame.shape
                    bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                           int(bboxC.width * iw), int(bboxC.height * ih)
                    x, y, w, h = bbox
                    
                    # Check if bbox is valid
                    if x < 0 or y < 0 or x + w > iw or y + h > ih:
                        continue  # Skip to the next frame
                    
                    face_roi = frame[y:y+h, x:x+w]

                    # Check if face_roi is valid
                    if face_roi.size == 0:
                        continue  # Skip to the next frame

                    try:
                        rgb_face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
                        face_mesh_results = face_mesh.process(rgb_face_roi)
                    except cv2.error as e:
                        print(f"OpenCV error: {e}")
                        continue  # Skip to the next frame

                    if face_mesh_results.multi_face_landmarks:
                        for face_landmarks in face_mesh_results.multi_face_landmarks:
                            landmarks = [(lm.x, lm.y) for lm in face_landmarks.landmark]
                            landmarks = np.array(landmarks)

                            # Update indices to use 468 landmarks
                            left_eye = landmarks[36:42]
                            right_eye = landmarks[42:48]
                            mouth = landmarks[48:68]
                            jawline = landmarks[0:17]  # Jawline (from 0 to 16)

                            left_ear = eye_aspect_ratio(left_eye)
                            right_ear = eye_aspect_ratio(right_eye)
                            ear = (left_ear + right_ear) / 2.0

                            mar = calculate_mar(mouth)

                            if ear < EYE_AR_THRESH:
                                blink_frames += 1
                            else:
                                if blink_frames >= EYE_AR_CONSEC_FRAMES:
                                    blink_count += 1
                                blink_frames = 0

                            if mar > MOUTH_AR_THRESH:
                                yawn_frames += 1
                            else:
                                if yawn_frames >= MOUTH_AR_CONSEC_FRAMES:
                                    yawn_count += 1
                                    if yawn_count > last_yawn_count:
                                        increasing_yawn_streak += 1
                                    else:
                                        increasing_yawn_streak = 0
                                yawn_frames = 0

                            last_yawn_count = yawn_count

                            if increasing_yawn_streak >= YAWN_ALERT_CONSEC_FRAMES:
                                warning_message = 'FATIGUE'
                                warning_start_time = time.time()
                                speak("You look fatigued, please take a break.")
                                increasing_yawn_streak = 0

                            jaw_center_x = np.mean(jawline[:, 0] * iw)

                            horizontal_threshold = 50

                            if jaw_center_x < center_x - horizontal_threshold:
                                position = 'left'
                                if head_position_start_time is None:
                                    head_position_start_time = time.time()
                            elif jaw_center_x > center_x + horizontal_threshold:
                                position = 'right'
                                if head_position_start_time is None:
                                    head_position_start_time = time.time()
                            else:
                                position = 'center'
                                head_position_start_time = None

                            if head_position_start_time and time.time() - head_position_start_time >= HEAD_POSITION_ALERT_SECONDS:
                                warning_message = 'Look straight'
                                warning_start_time = time.time()
                                speak("Please look straight.")
                                head_position_start_time = None

            current_time = time.time()

            # Update and display counts every 30 seconds
            if current_time - count_update_time >= COUNT_INTERVAL:
                count_update_time = current_time

            if current_time - last_blink_update_time >= DROWSY_ALERT_SECONDS:
                warning_message = 'DROWSY'
                warning_start_time = time.time()
                speak("You appear drowsy, please stay alert.")
                last_blink_update_time = current_time

            if warning_message and current_time - warning_start_time < WARNING_DISPLAY_SECONDS:
                text_x = max(frame_width - 400, 10)
                text_y = 100
                cv2.putText(frame, warning_message, (text_x, text_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)  # Smaller text size

            # Display blink and yawn counts
            cv2.putText(frame, f'Blinks: {blink_count}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f'Yawns: {yawn_count}', (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f'Position: {position}', (10, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Get current time and date
            current_time = datetime.now().strftime('%H:%M:%S')
            current_date = datetime.now().strftime('%d-%m-%Y')

            # Combine time and date
            time_date_text = f"{current_time}\n{current_date}"

            # Get the text size to calculate the position at the bottom right
            text_size_time = cv2.getTextSize(current_time, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
            text_size_date = cv2.getTextSize(current_date, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]

            # Calculate positions with added space between time and date
            time_position = (frame_width - text_size_time[0] - 10, frame_height - text_size_date[1] - 50)  # Added space
            date_position = (frame_width - text_size_date[0] - 10, frame_height - 10)

            # Display the current time and date on the frame in red color
            cv2.putText(frame, current_time, time_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, current_date, date_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.imshow('Face Landmarks Detection', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

cap.release()
cv2.destroyAllWindows()


In [2]:
# this head position code is perfect
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance
import time
from datetime import datetime
import pyttsx3
import threading
import requests

# Initialize Mediapipe components
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh

# Initialize pyttsx3 engine for voice alerts
engine = pyttsx3.init()

def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def calculate_mar(mouth):
    A = distance.euclidean(mouth[2], mouth[6])  # Vertical distance
    B = distance.euclidean(mouth[3], mouth[5])  # Vertical distance
    C = distance.euclidean(mouth[0], mouth[4])  # Horizontal distance
    return (A + B) / (2.0 * C)

# Fetch location using IP-based service
def get_current_location():
    ip_info_url = "http://ipinfo.io/json"
    response = requests.get(ip_info_url)
    data = response.json()
    
    city = data.get('city', 'Unknown City')
    region = data.get('region', 'Unknown Region')
    country = data.get('country', 'Unknown Country')
    
    location = f"{city}, {region}, {country}"
    
    return location

# Get current location once at the start
location_text = get_current_location()

# Define constants
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
MOUTH_AR_THRESH = 0.7
MOUTH_AR_CONSEC_FRAMES = 5
YAWN_ALERT_CONSEC_FRAMES = 3
DROWSY_ALERT_SECONDS = 20
WARNING_DISPLAY_SECONDS = 10
HEAD_POSITION_ALERT_SECONDS = 5
COUNT_INTERVAL = 30  # Interval to update and display counts

# Initialize counters and time tracking
blink_count = 0
yawn_count = 0
blink_frames = 0
yawn_frames = 0
consec_yawn_frames = 0
last_yawn_count = 0
increasing_yawn_streak = 0
last_blink_update_time = time.time()
count_update_time = time.time()  # Timer for counting interval

warning_message = ""
warning_start_time = 0
head_position_start_time = None
position = 'center'

# Open video capture with 720p resolution
cap = cv2.VideoCapture(0)

# Set the resolution and frame rate to 720p
width = 1280
height = 720
fps = 30

cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FPS, fps)

cv2.namedWindow('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True) as face_mesh:
    with mp_face_detection.FaceDetection(min_detection_confidence=0.1) as face_detection:

        def speak(message):
            def speak_thread():
                engine.say(message)
                engine.runAndWait()
            
            threading.Thread(target=speak_thread).start()

        last_update_time = time.time()
        update_interval = 3  # Update every 3 seconds
        display_index = 0

        # Define landmarks for left and right eyes
        LEFT_EYE = [33, 160, 158, 133, 153, 144]
        RIGHT_EYE = [362, 385, 387, 263, 373, 380]

        # Define thresholds for head position detection
        horizontal_threshold = 120
        vertical_threshold = 90

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            frame_height, frame_width = frame.shape[:2]
            center_x, center_y = frame_width // 2, frame_height // 2
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_detection.process(rgb_frame)

            if results.detections:
                for detection in results.detections:
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = frame.shape
                    bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                           int(bboxC.width * iw), int(bboxC.height * ih)
                    x, y, w, h = bbox
                    
                    if x < 0 or y < 0 or x + w > iw or y + h > ih:
                        continue
                    
                    face_roi = frame[y:y+h, x:x+w]

                    if face_roi.size == 0:
                        continue

                    try:
                        rgb_face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
                        face_mesh_results = face_mesh.process(rgb_face_roi)
                    except cv2.error as e:
                        print(f"OpenCV error: {e}")
                        continue

                    if face_mesh_results.multi_face_landmarks:
                        for face_landmarks in face_mesh_results.multi_face_landmarks:
                            landmarks = [(lm.x, lm.y) for lm in face_landmarks.landmark]
                            landmarks = np.array(landmarks)

                            left_eye = landmarks[36:42]
                            right_eye = landmarks[42:48]
                            mouth = landmarks[48:68]
                            jawline = landmarks[0:17]  # Jawline (from 0 to 16)

                            left_ear = eye_aspect_ratio(left_eye)
                            right_ear = eye_aspect_ratio(right_eye)
                            ear = (left_ear + right_ear) / 2.0

                            mar = calculate_mar(mouth)

                            if ear < EYE_AR_THRESH:
                                blink_frames += 1
                            else:
                                if blink_frames >= EYE_AR_CONSEC_FRAMES:
                                    blink_count += 1
                                blink_frames = 0

                            if mar > MOUTH_AR_THRESH:
                                yawn_frames += 1
                            else:
                                if yawn_frames >= MOUTH_AR_CONSEC_FRAMES:
                                    yawn_count += 1
                                    if yawn_count > last_yawn_count:
                                        increasing_yawn_streak += 1
                                    else:
                                        increasing_yawn_streak = 0
                                yawn_frames = 0

                            last_yawn_count = yawn_count

                            if increasing_yawn_streak >= YAWN_ALERT_CONSEC_FRAMES:
                                warning_message = 'You are fatigued, please be alert.'
                                warning_start_time = time.time()
                                speak("You look fatigued, please take a break.")
                                increasing_yawn_streak = 0

                            # Calculate jawline center for head position detection
                            jaw_center_x = np.mean(jawline[:, 0] * iw)
                            jaw_center_y = np.mean(jawline[:, 1] * ih)

                            # Debug prints for jawline center
                            print(f"Jawline center X: {jaw_center_x}")
                            print(f"Jawline center Y: {jaw_center_y}")

                            # Head position detection based on jawline center
                            if jaw_center_x < center_x - horizontal_threshold:
                                position = 'left'
                                if head_position_start_time is None:
                                    head_position_start_time = time.time()
                            elif jaw_center_x > center_x + horizontal_threshold:
                                position = 'right'
                                if head_position_start_time is None:
                                    head_position_start_time = time.time()
                            elif jaw_center_y < center_y - vertical_threshold:
                                position = 'up'
                                if head_position_start_time is None:
                                    head_position_start_time = time.time()
                            elif jaw_center_y > center_y + vertical_threshold:
                                position = 'down'
                                if head_position_start_time is None:
                                    head_position_start_time = time.time()
                            else:
                                position = 'center'
                                head_position_start_time = None

                            # Debug print for detected position
                            print(f"Detected position: {position}")

                            # Only trigger alert if the head is turned for the entire duration
                            if head_position_start_time and time.time() - head_position_start_time >= HEAD_POSITION_ALERT_SECONDS:
                                if position in ['left']:
                                    warning_message = "Please look straight"
                                    warning_start_time = time.time()
                                    speak(f"Please look straight you are turning left.")
                                head_position_start_time = None
                                
                                if position in ['right']:
                                    warning_message = "Please look straight"
                                    warning_start_time = time.time()
                                    speak(f"Please look straight you are turning right.")
                                head_position_start_time = None
                                
                                if position in ['up']:
                                    warning_message = "Please look straight"
                                    warning_start_time = time.time()
                                    speak(f"Please look straight you are facing up.")
                                head_position_start_time = None
                                
            current_time = time.time()

            # Update and display counts every 30 seconds
            if current_time - count_update_time >= COUNT_INTERVAL:
                count_update_time = current_time

            # Trigger drowsiness alert if no blinking for a long duration
            if current_time - last_blink_update_time >= DROWSY_ALERT_SECONDS:
                warning_message = 'You are drowsy, please stay alert!'
                warning_start_time = current_time
                speak("You are drowsy, please stay alert.")
                
                # Reset counters
                blink_count = 0
                yawn_count = 0

            # Check if warning message should be updated
            if warning_message and current_time - warning_start_time <= WARNING_DISPLAY_SECONDS:
                cv2.putText(frame, warning_message, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                warning_message = ""

            # Display the blink and yawn counts on the video feed
            cv2.putText(frame, f"Blink Count: {blink_count}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, f"Yawn Count: {yawn_count}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, f"Location: {location_text}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Display head position message
            if position != 'center':
                cv2.putText(frame, f"Head Position: {position.capitalize()}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
            cv2.imshow('Face Landmarks Detection', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

cap.release()
cv2.destroyAllWindows()


Jawline center X: 634.5503728529986
Jawline center Y: 455.39602518081665
Detected position: down
Jawline center X: 651.2240802540499
Jawline center Y: 465.9354897106395
Detected position: down
Jawline center X: 664.927054012523
Jawline center Y: 469.0203696138719
Detected position: down
Jawline center X: 648.605759564568
Jawline center Y: 456.36215560576494
Detected position: down
Jawline center X: 648.5885934268726
Jawline center Y: 466.0983711130479
Detected position: down
Jawline center X: 645.4009089750402
Jawline center Y: 459.128743760726
Detected position: down
Jawline center X: 652.0300001256606
Jawline center Y: 456.750651808346
Detected position: down
Jawline center X: 634.2019025017233
Jawline center Y: 458.0812887584462
Detected position: down
Jawline center X: 630.8657186171588
Jawline center Y: 454.8031345535727
Detected position: down
Jawline center X: 632.6746458165786
Jawline center Y: 454.06377638087554
Detected position: down


# Blink code

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist

# Initialize MediaPipe Face Mesh and drawing utilities
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Eye Aspect Ratio (EAR) calculation
def calculate_ear(eye_landmarks):
    # Vertical eye landmarks (used for EAR calculation)
    vert1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vert2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    # Horizontal eye landmark
    horz = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    # EAR formula
    ear = (vert1 + vert2) / (2.0 * horz)
    return ear

# Indices for the left and right eye landmarks (from MediaPipe's 468 landmarks)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

# Thresholds and consecutive frame count for blink detection
EAR_THRESHOLD = 0.25
CONSECUTIVE_FRAMES = 2
blink_count = 0
frame_counter = 0

# Start the video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Flip the frame horizontally for a later selfie-view display
    frame = cv2.flip(frame, 1)
    
    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame with MediaPipe Face Mesh
    results = face_mesh.process(rgb_frame)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Extract left and right eye landmarks
            left_eye = np.array([(int(face_landmarks.landmark[i].x * frame.shape[1]),
                                  int(face_landmarks.landmark[i].y * frame.shape[0])) for i in LEFT_EYE])
            right_eye = np.array([(int(face_landmarks.landmark[i].x * frame.shape[1]),
                                   int(face_landmarks.landmark[i].y * frame.shape[0])) for i in RIGHT_EYE])
            
            # Calculate EAR for both eyes
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0
            
            # Check if EAR is below the threshold and increment frame counter
            if ear < EAR_THRESHOLD:
                frame_counter += 1
            else:
                if frame_counter >= CONSECUTIVE_FRAMES:
                    blink_count += 1
                    print("Blink detected!")
                frame_counter = 0
            
            # Draw the eye landmarks for visualization
            for point in left_eye:
                cv2.circle(frame, tuple(point), 2, (0, 255, 0), -1)
            for point in right_eye:
                cv2.circle(frame, tuple(point), 2, (0, 255, 0), -1)
    
    # Display the blink count
    cv2.putText(frame, f"Blinks: {blink_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Show the frame
    cv2.imshow('Blink Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


# Head and blink

In [4]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
import time
from datetime import datetime
import requests

# Initialize Mediapipe components
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Define constants for EAR and MAR thresholds
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
MOUTH_AR_THRESH = 0.7
MOUTH_AR_CONSEC_FRAMES = 5

# Indices for landmarks
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
MOUTH = list(range(48, 68))  # Full mouth landmarks
JAWLINE = list(range(0, 17))  # Jawline (from 0 to 16)

# Initialize counters and time tracking
blink_count = 0
yawn_count = 0
blink_frames = 0
yawn_frames = 0
position = 'center'

# Fetch location using IP-based service
def get_current_location():
    ip_info_url = "http://ipinfo.io/json"
    response = requests.get(ip_info_url)
    data = response.json()
    
    city = data.get('city', 'Unknown City')
    region = data.get('region', 'Unknown Region')
    country = data.get('country', 'Unknown Country')
    
    location = f"{city}, {region}, {country}"
    
    return location

location_text = get_current_location()

# Define EAR and MAR calculation functions
def calculate_ear(eye_landmarks):
    vert1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vert2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    horz = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (vert1 + vert2) / (2.0 * horz)
    return ear

def calculate_mar(mouth):
    A = dist.euclidean(mouth[2], mouth[6])  # Vertical distance
    B = dist.euclidean(mouth[3], mouth[5])  # Vertical distance
    C = dist.euclidean(mouth[0], mouth[4])  # Horizontal distance
    return (A + B) / (2.0 * C)

def get_head_position(jawline, frame_width, frame_height):
    # Calculate the average position of the jawline landmarks
    jawline_x = jawline[:, 0]
    jawline_y = jawline[:, 1]
    
    # Calculate average position for the jawline
    jaw_center_x = np.mean(jawline_x)
    jaw_center_y = np.mean(jawline_y)
    
    # Get the center of the frame
    center_x = frame_width / 2
    center_y = frame_height / 2
    
    # Define thresholds for detecting left, right, up, and down positions
    horizontal_threshold = 60  # Threshold for detecting horizontal movement
    vertical_threshold = 50    # Threshold for detecting vertical movement
    
    # Determine head position
    if jaw_center_x < center_x - horizontal_threshold:
        return 'left'
    elif jaw_center_x > center_x + horizontal_threshold:
        return 'right'
    elif jaw_center_y < center_y - vertical_threshold:
        return 'up'
    elif jaw_center_y > center_y + vertical_threshold:
        return 'down'
    else:
        return 'center'




# Open video capture with 720p resolution
cap = cv2.VideoCapture(0)
width = 1280
height = 720
fps = 30

cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FPS, fps)

cv2.namedWindow('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = np.array([(lm.x * frame_width, lm.y * frame_height) for lm in face_landmarks.landmark])

            # Extract landmarks
            left_eye = landmarks[LEFT_EYE]
            right_eye = landmarks[RIGHT_EYE]
            mouth = landmarks[MOUTH]
            jawline = landmarks[JAWLINE]

            # Calculate EAR and MAR
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = calculate_mar(mouth)

            # Blink detection logic
            if ear < EYE_AR_THRESH:
                blink_frames += 1
            else:
                if blink_frames >= EYE_AR_CONSEC_FRAMES:
                    blink_count += 1
                blink_frames = 0

            # Yawn detection logic
            if mar > MOUTH_AR_THRESH:
                yawn_frames += 1
            else:
                if yawn_frames >= MOUTH_AR_CONSEC_FRAMES:
                    yawn_count += 1
                yawn_frames = 0

            # Calculate head position
            position = get_head_position(jawline, frame_width, frame_height)

    # Display blink and yawn counts
    cv2.putText(frame, f'Blinks: {blink_count}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Yawns: {yawn_count}', (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display head position
    cv2.putText(frame, f'Head Position: {position.capitalize()}', (10, 110),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    # Display current date and time
    current_time = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    cv2.putText(frame, f'Time: {current_time}', (10, frame_height - 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    # Display current location
    cv2.putText(frame, f'Location: {location_text}', (10, frame_height - 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow('Face Landmarks Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Their code

In [2]:
# codeee
import cv2
import mediapipe as mp
import numpy as np
import time
from scipy.spatial import distance as dist
import winsound
from datetime import datetime

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

def toggle_fullscreen():
    # Toggle fullscreen mode
    if cv2.getWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN) == cv2.WINDOW_FULLSCREEN:
        cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)
    else:
        cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Create a named window
window_name = 'Driver State Analysis'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

# Start in fullscreen mode
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
cap = cv2.VideoCapture(0)

# Blink and Yawn Counters
blink_count = 0
yawn_count = 0
frequent_blink_alert_count = 0
frequent_yawn_alert_count = 0
head_position_change_alert_count=0


# Constants for blink and yawn detection
EYE_AR_THRESH = 0.2
EYE_AR_CONSEC_FRAMES = 2
MOUTH_AR_THRESH = 0.7
MOUTH_OPEN_CONSEC_FRAMES = 5

# Initialize counters for consecutive frames
COUNTER = 0
yawn_counter = 0

# Mouth open state flag
mouth_open = False

# Cooldown period for yawns in seconds
yawn_cooldown_period = 5
last_yawn_time = 0  # Tracks the last time a yawn was detected

# Timer for blink count reset and alert
start_time = time.time()
reset_interval = 30  # Reset interval in seconds (30 seconds)
blink_last_reset_time = start_time  
frequent_blink_alert_reset_time = start_time
#frequent_yawn_alert_reset_time = start_time
head_position_change_alert_reset_time= start_time
# New variables for frequent blinking alert
alert_duration = 5  # Duration for alert message in seconds
frequent_blink_alert_start_time = None  # Track the start time of the frequent blinking alert
frequent_blink_alert_shown = False  # Flag to control frequent blinking alert message display

# Variables for eyes closed alert
eyes_closed_start_time = None  # Track when the eyes were first closed
eyes_closed_alert_shown = False  # Flag to control the eyes closed alert message display
eyes_closed_alert_duration = 5  # Duration to show the eyes closed alert in seconds

# Timer for yawn count reset and alert
yawn_start_time = time.time()  # Timer to track when to reset the yawn count
frequent_yawn_alert_start_time = None  # Track the start time of the frequent yawning alert
frequent_yawn_alert_shown = False  # Flag to control frequent yawning alert message display
last_reset_time = None

# Variables for "Look at the road" alert
look_at_road_start_time = None  # Track the time when the person last looked forward
look_at_road_alert_shown = False  # Flag to control the "Look at the road" alert message display
look_at_road_alert_duration = 5  # Duration to show the "Look at the road" alert in seconds

previous_position = "Forward"
position_change_count = 0

# Timer for head position change reset and alert
head_position_change_start_time = time.time()  # Timer to track when to reset the position change count
head_position_change_last_reset_time = None  # Track the last reset time
head_position_change_alert_start_time = None  # Track the start time of the position change alert
head_position_change_alert_shown = False  # Flag to control position change alert message display
head_position_change_alert_duration = 5  # Duration to show the position change alert in seconds


current_state = "Awake"

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[0], mouth[1])  # Upper lip to lower lip
    C = dist.euclidean(mouth[2], mouth[3])  # Left corner to right corner of the mouth
    mar = A / C
    return mar

while cap.isOpened():
    success, image = cap.read()
    current_time = time.time()
    start = time.time()

    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = face_mesh.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = image.shape
    face_3d = []
    face_2d = []

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = face_landmarks.landmark

            # Correct indices for eyes
            left_eye = [landmarks[i] for i in [33, 160, 158, 133, 153, 144]]
            right_eye = [landmarks[i] for i in [362, 385, 387, 263, 373, 380]]

            left_eye = np.array([(lm.x * img_w, lm.y * img_h) for lm in left_eye], dtype=np.float64)
            right_eye = np.array([(lm.x * img_w, lm.y * img_h) for lm in right_eye], dtype=np.float64)

            left_ear = eye_aspect_ratio(left_eye)
            right_ear = eye_aspect_ratio(right_eye)
            ear = (left_ear + right_ear) / 2.0

            # Correct indices for mouth landmarks
            mouth = [landmarks[i] for i in [13, 14, 78, 308]]
            mouth = np.array([(lm.x * img_w, lm.y * img_h) for lm in mouth], dtype=np.float64)

            mar = mouth_aspect_ratio(mouth)

            # Head pose estimation
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)

                    x, y = int(lm.x * img_w), int(lm.y * img_h)
                    face_2d.append([x, y])
                    face_3d.append([x, y, lm.z])

            face_2d = np.array(face_2d, dtype=np.float64)
            face_3d = np.array(face_3d, dtype=np.float64)

            focal_length = 1 * img_w

            cam_matrix = np.array([[focal_length, 0, img_h / 2],
                                   [0, focal_length, img_w / 2],
                                   [0, 0, 1]])

            dist_matrix = np.zeros((4, 1), dtype=np.float64)

            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

            rmat, jac = cv2.Rodrigues(rot_vec)
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360

            # Determine head pose and use corresponding eye aspect ratio threshold
            if y < -17:
                current_position = "Looking Left"
            elif y > 17:
                current_position = "Looking Right"
            elif x < -15:
                current_position = "Looking Down"
            elif x > 19:
                current_position= "Looking Up"
            else:
                current_position = "Forward"

                # Update look_at_road_start_time when looking forward
                look_at_road_start_time = current_time  # Reset the timer when looking forward
                look_at_road_alert_shown = False  # Reset the alert flag

           # Blink detection logic using the current threshold
            if ear < EYE_AR_THRESH:
                COUNTER += 1
                if eyes_closed_start_time is None:
                    eyes_closed_start_time = time.time()
                elif time.time() - eyes_closed_start_time > 5:  # Eyes closed for more than 5 seconds
                    if not eyes_closed_alert_shown:
                        eyes_closed_alert_shown = True
                        eyes_closed_alert_start_time = time.time()
                        
            else:
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    blink_count += 1
                COUNTER = 0
                eyes_closed_start_time = None  # Reset the eyes closed start time

            # Yawn detection logic with the new reset mechanism
            current_time = time.time()
            if mar > MOUTH_AR_THRESH:
                yawn_counter += 1
                if yawn_counter >= MOUTH_OPEN_CONSEC_FRAMES and not mouth_open:
                    if current_time - last_yawn_time > yawn_cooldown_period:
                        yawn_count += 1
                        mouth_open = True
                        last_yawn_time = current_time
                        
            else:
                mouth_open = False
                yawn_counter = 0

               # Yawn count reset logic with a delay
            if yawn_count >= 5:
                if last_reset_time is None:
                    last_reset_time = current_time  # Track the time when count reaches 5
                elif current_time - last_reset_time >= 5:  # Wait for 2 seconds if last_reset_time is set
                    yawn_count = 0  # Reset the count
                    yawn_start_time = current_time  # Start a new 1-minute interval
                    last_reset_time = None  # Reset the last_reset_time for future use
            elif current_time - yawn_start_time >= 60:
                yawn_count = 0
                yawn_start_time = current_time

                    # Check for position change (ignore changes to 'Forward')
            if current_position != "Forward" and current_position != previous_position:
                position_change_count += 1
                previous_position = current_position  # Update the previous position

            # Head position change count reset logic with a delay
            if position_change_count >= 7:
                if head_position_change_last_reset_time is None:
                    head_position_change_last_reset_time = current_time  # Track the time when count reaches 7
                elif current_time - head_position_change_last_reset_time >= 5:  # Wait for 5 seconds if last_reset_time is set
                    position_change_count = 0  # Reset the count
                    head_position_change_start_time = current_time  # Start a new 1-minute interval
                    head_position_change_last_reset_time = None  # Reset the last_reset_time for future use
            elif current_time - head_position_change_start_time >= 60:
                position_change_count = 0
                head_position_change_start_time = current_time



            nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)

            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            p2 = (int(nose_2d[0] + y * 10), int(nose_2d[1] - x * 10))
            
            #cv2.line(image, p1, p2, (255, 0, 0), 3)

            # Check if the blink count exceeds 12 or 24
            # Check if the blink count exceeds 12
            if blink_count >= 13:
                blink_count = 0  # Reset immediately
                blink_last_reset_time = time.time()  # Start a new 30-second interval
            elif time.time() - blink_last_reset_time >= reset_interval:
                blink_count = 0
                blink_last_reset_time = time.time()  # Start a new 30-second interval
            
            if blink_count in [12, 24]:
                if not frequent_blink_alert_shown:
                    frequent_blink_alert_start_time = time.time() 
                    frequent_blink_alert_count += 1 # Start the frequent blinking alert
                    frequent_blink_alert_shown = True

            # Display frequent blinking alert message if needed
            if frequent_blink_alert_shown:
                if time.time() - frequent_blink_alert_start_time <= alert_duration:  # Show the alert for the specified duration
                    cv2.putText(image, "Frequent Blinking Alert", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                    cv2.putText(image, "DROWSY", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 2)
                else:
                    frequent_blink_alert_shown = False  # Reset the flag to stop showing the alert
                    
            # Display eyes closed for too long alert message if needed
            if eyes_closed_start_time is not None:
                elapsed_time = time.time() - eyes_closed_start_time
                if elapsed_time > eyes_closed_alert_duration:
                    eyes_closed_alert_shown = True
            else:
                    eyes_closed_alert_shown = False
                    #current_state="Awake"

            # Display eye closure alert message if needed
            if eyes_closed_alert_shown:
                cv2.putText(image, "WAKE UP! WAKE UP!", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                current_state= "Drowsy"
                cv2.putText(image, "DROWSY", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 2)
                winsound.Beep(2000,500)
            else :
                eyes_closed_alert_shown = False
            if yawn_count >= 5:
                if not frequent_yawn_alert_shown:
                    frequent_yawn_alert_start_time = time.time() 
                    #frequent_yawn_alert_count += 1 # Start the frequent yawning alert
                    frequent_yawn_alert_shown = True
    # Display frequent yawning alert message if needed
                if frequent_yawn_alert_shown:
                        if time.time() - frequent_yawn_alert_start_time <= alert_duration:  # Show the alert for the specified duration
                            cv2.putText(image, "Frequent Yawning Detected!", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                            cv2.putText(image, "FATIGUE", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 2)
                        else:
                            frequent_yawn_alert_shown = False
                             # Stop showing the alert message
            else:
    # Reset the alert state if yawn count goes below 5
                frequent_yawn_alert_shown = False

            if time.time() - start_time >= reset_interval:
                blink_count = 0
                start_time = time.time()
            # Check if the person has not looked forward for more than 15 seconds
            if look_at_road_start_time is not None and current_time - look_at_road_start_time > 3:
                if not look_at_road_alert_shown:
                    look_at_road_alert_shown = True
                    look_at_road_alert_start_time = current_time

# Display the "Look at the road" alert message if needed
            if look_at_road_alert_shown:
                if time.time() - look_at_road_alert_start_time <= look_at_road_alert_duration:
                    cv2.putText(image, "Look at the Road!", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(image, "VISUAL COGNITIVE", (100, 400), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 2)
                    winsound.Beep(2000,100)
                else:
                    look_at_road_alert_shown = False 
                    current_state="Awake" # Stop showing the alert message
            # Check if the head position change count reaches 7
            if position_change_count >= 7:
                if not head_position_change_alert_shown:
                    head_position_change_alert_start_time = time.time()  # Start the head position change alert
                    head_position_change_alert_count+=1
                    
                    head_position_change_alert_shown = True

# Display head position change alert message if needed
            if head_position_change_alert_shown:
                if time.time() - head_position_change_alert_start_time <= head_position_change_alert_duration:  # Show the alert for the specified duration
                    cv2.putText(image, "Distraction Detected!", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(image, "VISUAL COGNITIVE", (100, 400), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 2)
                else:
                    head_position_change_alert_shown = False  # Stop showing the alert message

 

            # Reset the frequent blinking alert count every minute
            if current_time - frequent_blink_alert_reset_time >= 60 or eyes_closed_alert_shown :
               frequent_blink_alert_count = 0  # Reset the frequent blink alert count
               frequent_blink_alert_reset_time = time.time()
               
            #if frequent_blink_alert_count >= 2:
            #    current_state = "Drowsy"  
            # Update the reset time  

            #if head_position_change_alert_count>= 2:
            #    current_state = "Visual Cognitive"  

            #if current_time - frequent_yawn_alert_reset_time >= 120:
            #    frequent_yawn_alert_count = 0  # Reset the frequent blink alert count
            #    frequent_yawn_alert_reset_time = time.time()  # Update the reset time  

            # Reset the frequent blinking alert count every minute
            if current_time - head_position_change_alert_reset_time >= 60 or head_position_change_alert_shown:
                head_position_change_alert_count= 0  # Reset the frequent blink alert count
                head_position_change_alert_reset_time = time.time()  # Update the reset time              
            #cv2.putText(image, f"Blink Count: {blink_count}", (30, 150),
                        #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 100, 100), 2)
            #cv2.putText(image, f"Yawn Count: {yawn_count}", (30, 180),
                        #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (100, 250, 0), 2)
            #cv2.putText(image, f"Head Pose: {current_position}", (30, 210),
                        #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 105, 0), 2)
            #cv2.putText(image, f"Head Position Change Count: {position_change_count}", (30, 240),
                    #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 100, 0), 2)
            #cv2.putText(image, f"Frequent Blink Alerts: {frequent_blink_alert_count}", (30, 280), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            #cv2.putText(image, f"Frequent Yawn Alerts: {frequent_yawn_alert_count}", (30, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            #cv2.putText(image, f"Head Position Change Alerts: {head_position_change_alert_count}", (30, 320), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            #cv2.putText(image, f'State: {current_state}', (10, img_h - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    end = time.time()

    totalTime = end - start

    if totalTime > 0:
                fps = 1 / totalTime
    else:
                fps = 0  # Set fps to 0 or handle this case as needed
    #cv2.putText(image, f'FPS: {int(fps)}', (500, 450), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Draw the face mesh landmarks (all 468 points)
    #mp_drawing.draw_landmarks(
                #image=image,
                #landmark_list=face_landmarks,
                #connections=mp_face_mesh.FACEMESH_CONTOURS,
                #landmark_drawing_spec=drawing_spec,
                #connection_drawing_spec=drawing_spec
#            )

    #cv2.putText(image, f'FPS: {int(fps)}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


    current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    cv2.putText(image,f"{current_datetime}",(10,img_h - 10),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),2)

    cv2.imshow(window_name, image)
    
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

# Pupil detection

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist

# Initialize Mediapipe components
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)

# Define constants
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
EYE_AR_THRESH = 0.25  # Adjust this threshold as needed
EYE_AR_CONSEC_FRAMES = 3

# Function to calculate pupil center
def get_pupil_center(eye_landmarks):
    x_coords = [pt[0] for pt in eye_landmarks]
    y_coords = [pt[1] for pt in eye_landmarks]
    center_x = int(sum(x_coords) / len(x_coords))
    center_y = int(sum(y_coords) / len(y_coords))
    return (center_x, center_y)

# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye_landmarks):
    # Example distances; adjust as needed for accuracy
    vert1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vert2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    horz = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (vert1 + vert2) / (2.0 * horz)
    return ear

# Initialize blink detection variables
blink_count = 0
blink_frames = 0

# Open video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = np.array([(lm.x * frame_width, lm.y * frame_height) for lm in face_landmarks.landmark])

            # Extract eye landmarks
            left_eye = landmarks[LEFT_EYE]
            right_eye = landmarks[RIGHT_EYE]

            # Calculate pupil centers
            pupil_left = get_pupil_center(left_eye)
            pupil_right = get_pupil_center(right_eye)

            # Calculate EAR for both eyes
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0

            # Blink detection logic
            if ear < EYE_AR_THRESH:
                blink_frames += 1
            else:
                if blink_frames >= EYE_AR_CONSEC_FRAMES:
                    blink_count += 1
                blink_frames = 0

            # Draw pupil marks
            cv2.circle(frame, pupil_left, 5, (0, 255, 0), -1)
            cv2.circle(frame, pupil_right, 5, (0, 255, 0), -1)

            # Display blink count
            cv2.putText(frame, f'Blinks: {blink_count}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Pupil Detection and Blink Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Head position and blink with voice command

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
import requests
from datetime import datetime
import time
import pyttsx3
import threading

# Initialize Mediapipe components
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define constants for EAR and MAR thresholds
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
MOUTH_AR_THRESH = 0.5  # Adjusted threshold
MOUTH_AR_CONSEC_FRAMES = 15  # Adjusted for better detection
DROWSINESS_TIME_THRESHOLD = 5  # 5 seconds for drowsiness detection
HEAD_POSITION_TIME_THRESHOLD = 10  # 10 seconds for head position alert

# Indices for landmarks
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
MOUTH = [78, 95, 88, 191, 80, 82, 13, 311, 308, 402, 317, 14]  # Updated for MediaPipe
JAWLINE = list(range(0, 17))  # Jawline (from 0 to 16)

# Initialize counters and time tracking
blink_count = 0
yawn_count = 0
blink_frames = 0
yawn_frames = 0
position = 'center'
eyes_closed_time = None
drowsy_command_played = False
head_position_start_time = None
look_at_road_command_played = False

# Fetch location using IP-based service
def get_current_location():
    ip_info_url = "http://ipinfo.io/json"
    response = requests.get(ip_info_url)
    data = response.json()
    
    city = data.get('city', 'Unknown City')
    region = data.get('region', 'Unknown Region')
    country = data.get('country', 'Unknown Country')
    
    location = f"{city}, {region}, {country}"
    
    return location

location_text = get_current_location()

# Define EAR and MAR calculation functions
def calculate_ear(eye_landmarks):
    vert1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vert2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    horz = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (vert1 + vert2) / (2.0 * horz)
    return ear

def calculate_mar(mouth_landmarks):
    A = dist.euclidean(mouth_landmarks[1], mouth_landmarks[7])  # Upper to lower lip
    B = dist.euclidean(mouth_landmarks[3], mouth_landmarks[5])  # Left to right corner
    mar = A / B
    return mar

def get_head_position(jawline, frame_width, frame_height):
    jawline_x = jawline[:, 0]
    jawline_y = jawline[:, 1]
    
    jaw_center_x = np.mean(jawline_x)
    jaw_center_y = np.mean(jawline_y)
    
    center_x = frame_width / 2
    center_y = frame_height / 2
    
    horizontal_threshold = 60  # Threshold for detecting horizontal movement
    vertical_threshold = 50    # Threshold for detecting vertical movement
    
    if jaw_center_x < center_x - horizontal_threshold:
        return 'left'
    elif jaw_center_x > center_x + horizontal_threshold:
        return 'right'
    elif jaw_center_y < center_y - vertical_threshold:
        return 'up'
    elif jaw_center_y > center_y + vertical_threshold:
        return 'down'
    else:
        return 'center'

# Function to calculate pupil center
def get_pupil_center(eye_landmarks):
    x_coords = [pt[0] for pt in eye_landmarks]
    y_coords = [pt[1] for pt in eye_landmarks]
    center_x = int(sum(x_coords) / len(x_coords))
    center_y = int(sum(y_coords) / len(y_coords))
    return (center_x, center_y)

# Function to play voice command
def play_voice_command(message):
    engine.say(message)
    engine.runAndWait()

# Function to run voice command in a separate thread
def thread_play_voice_command(message):
    t = threading.Thread(target=play_voice_command, args=(message,))
    t.start()

# Open video capture with 720p resolution
cap = cv2.VideoCapture(0)
width = 1280
height = 720
fps = 30

cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FPS, fps)

cv2.namedWindow('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = np.array([(lm.x * frame_width, lm.y * frame_height) for lm in face_landmarks.landmark])

            # Extract landmarks
            left_eye = landmarks[LEFT_EYE]
            right_eye = landmarks[RIGHT_EYE]
            mouth = landmarks[MOUTH]
            jawline = landmarks[JAWLINE]

            # Calculate EAR and MAR
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = calculate_mar(mouth)

            # Blink detection logic
            if ear < EYE_AR_THRESH:
                blink_frames += 1

                # Start counting the duration of eye closure
                if eyes_closed_time is None:
                    eyes_closed_time = time.time()

                # Check if eyes are closed for more than the threshold
                if time.time() - eyes_closed_time >= DROWSINESS_TIME_THRESHOLD:
                    if not drowsy_command_played:
                        thread_play_voice_command("You are feeling drowsy! Stay alert")
                        drowsy_command_played = True

            else:
                if blink_frames >= EYE_AR_CONSEC_FRAMES:
                    blink_count += 1

                blink_frames = 0
                eyes_closed_time = None
                drowsy_command_played = False

            # Yawn detection logic
            if mar > MOUTH_AR_THRESH:
                yawn_frames += 1
            else:
                if yawn_frames >= MOUTH_AR_CONSEC_FRAMES:
                    yawn_count += 1
                yawn_frames = 0

            # Calculate head position
            position = get_head_position(jawline, frame_width, frame_height)

            # Check head position for voice command
            if position != 'center':
                if head_position_start_time is None:
                    head_position_start_time = time.time()

                if time.time() - head_position_start_time >= HEAD_POSITION_TIME_THRESHOLD:
                    # Continuously play the voice command if head is not in the center
                    thread_play_voice_command("Look at the road")
                    look_at_road_command_played = True

            else:
                head_position_start_time = None
                look_at_road_command_played = False

            # Calculate pupil centers
            pupil_left = get_pupil_center(left_eye)
            pupil_right = get_pupil_center(right_eye)

            # Draw pupil marks (hide these by commenting out the lines if not needed)
            # cv2.circle(frame, pupil_left, 5, (0, 255, 0), -1)
            # cv2.circle(frame, pupil_right, 5, (0, 255, 0), -1)

    # Display blink and yawn counts
    cv2.putText(frame, f'Blinks: {blink_count}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Yawns: {yawn_count}', (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display head position
    cv2.putText(frame, f'Head Position: {position.capitalize()}', (10, 110),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    # Display current date and time
    current_time = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    cv2.putText(frame, f'Time: {current_time}', (10, frame_height - 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    # Display current location
    cv2.putText(frame, f'Location: {location_text}', (10, frame_height - 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow('Face Landmarks Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
import requests
from datetime import datetime
import time
import pyttsx3
import threading

# Initialize Mediapipe components
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define constants for EAR and MAR thresholds
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
MOUTH_AR_THRESH = 0.5  # Adjusted threshold
MOUTH_AR_CONSEC_FRAMES = 15  # Adjusted for better detection
DROWSINESS_TIME_THRESHOLD = 5  # 5 seconds for drowsiness detection
HEAD_POSITION_TIME_THRESHOLD = 10  # 10 seconds for head position alert

# Indices for landmarks
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
MOUTH = [78, 95, 88, 191, 80, 82, 13, 311, 308, 402, 317, 14]  # Updated for MediaPipe
JAWLINE = list(range(0, 17))  # Jawline (from 0 to 16)

# Initialize counters and time tracking
blink_count = 0
yawn_count = 0
blink_frames = 0
yawn_frames = 0
position = 'center'
eyes_closed_time = None
drowsy_command_played = False
head_position_start_time = None
look_at_road_command_played = False

# Fetch location using IP-based service
def get_current_location():
    ip_info_url = "http://ipinfo.io/json"
    response = requests.get(ip_info_url)
    data = response.json()
    
    city = data.get('city', 'Unknown City')
    region = data.get('region', 'Unknown Region')
    country = data.get('country', 'Unknown Country')
    
    location = f"{city}, {region}, {country}"
    
    return location

location_text = get_current_location()

# Define EAR and MAR calculation functions
def calculate_ear(eye_landmarks):
    vert1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vert2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    horz = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (vert1 + vert2) / (2.0 * horz)
    return ear

def calculate_mar(mouth_landmarks):
    A = dist.euclidean(mouth_landmarks[1], mouth_landmarks[7])  # Upper to lower lip
    B = dist.euclidean(mouth_landmarks[3], mouth_landmarks[5])  # Left to right corner
    mar = A / B
    return mar

def get_head_position(jawline, frame_width, frame_height):
    jawline_x = jawline[:, 0]
    jawline_y = jawline[:, 1]
    
    jaw_center_x = np.mean(jawline_x)
    jaw_center_y = np.mean(jawline_y)
    
    center_x = frame_width / 2
    center_y = frame_height / 2
    
    horizontal_threshold = 60  # Threshold for detecting horizontal movement
    vertical_threshold = 50    # Threshold for detecting vertical movement
    
    if jaw_center_x < center_x - horizontal_threshold:
        return 'left'
    elif jaw_center_x > center_x + horizontal_threshold:
        return 'right'
    elif jaw_center_y < center_y - vertical_threshold:
        return 'up'
    elif jaw_center_y > center_y + vertical_threshold:
        return 'down'
    else:
        return 'center'

# Function to calculate pupil center
def get_pupil_center(eye_landmarks):
    x_coords = [pt[0] for pt in eye_landmarks]
    y_coords = [pt[1] for pt in eye_landmarks]
    center_x = int(sum(x_coords) / len(x_coords))
    center_y = int(sum(y_coords) / len(y_coords))
    return (center_x, center_y)

# Function to play voice command
def play_voice_command(message):
    engine.say(message)
    engine.runAndWait()

# Function to run voice command in a separate thread
def thread_play_voice_command(message):
    t = threading.Thread(target=play_voice_command, args=(message,))
    t.start()

# Open video capture with 720p resolution
cap = cv2.VideoCapture(0)
width = 1280
height = 720
fps = 30

cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FPS, fps)

cv2.namedWindow('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = np.array([(lm.x * frame_width, lm.y * frame_height) for lm in face_landmarks.landmark])

            # Extract landmarks
            left_eye = landmarks[LEFT_EYE]
            right_eye = landmarks[RIGHT_EYE]
            mouth = landmarks[MOUTH]
            jawline = landmarks[JAWLINE]

            # Calculate EAR and MAR
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = calculate_mar(mouth)

            # Blink detection logic
            if ear < EYE_AR_THRESH:
                blink_frames += 1

                # Start counting the duration of eye closure
                if eyes_closed_time is None:
                    eyes_closed_time = time.time()

                # Check if eyes are closed for more than the threshold
                if time.time() - eyes_closed_time >= DROWSINESS_TIME_THRESHOLD:
                    if not drowsy_command_played:
                        thread_play_voice_command("You are feeling drowsy! Stay alert")
                        drowsy_command_played = True

            else:
                if blink_frames >= EYE_AR_CONSEC_FRAMES:
                    blink_count += 1

                blink_frames = 0
                eyes_closed_time = None
                drowsy_command_played = False

            # Yawn detection logic
            if mar > MOUTH_AR_THRESH:
                yawn_frames += 1
            else:
                if yawn_frames >= MOUTH_AR_CONSEC_FRAMES:
                    yawn_count += 1
                yawn_frames = 0

            # Calculate head position
            position = get_head_position(jawline, frame_width, frame_height)

            # Check head position for voice command
            if position != 'center':
                if head_position_start_time is None:
                    head_position_start_time = time.time()

                if time.time() - head_position_start_time >= HEAD_POSITION_TIME_THRESHOLD:
                    if not look_at_road_command_played:
                        thread_play_voice_command("Look at the road")
                        look_at_road_command_played = True

            else:
                head_position_start_time = None
                look_at_road_command_played = False

            # Calculate pupil centers
            pupil_left = get_pupil_center(left_eye)
            pupil_right = get_pupil_center(right_eye)

            # Draw pupil marks
            cv2.circle(frame, pupil_left, 5, (0, 255, 0), -1)
            cv2.circle(frame, pupil_right, 5, (0, 255, 0), -1)

    # Display blink and yawn counts
    cv2.putText(frame, f'Blinks: {blink_count}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Yawns: {yawn_count}', (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display head position
    cv2.putText(frame, f'Head Position: {position.capitalize()}', (10, 110),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    # Display current date and time
    current_time = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    cv2.putText(frame, f'Time: {current_time}', (10, frame_height - 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    # Display current location
    cv2.putText(frame, f'Location: {location_text}', (10, frame_height - 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow('Face Landmarks Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\DELL\intern_proj\NEWN\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


# Yawn detection

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Function to calculate the Mouth Aspect Ratio (MAR)
def calculate_mar(landmarks):
    top_lip = landmarks[13:15]  # Top lip points
    bottom_lip = landmarks[14:16]  # Bottom lip points

    # Calculate the vertical distance
    vertical_distance = np.linalg.norm(np.array(top_lip[0]) - np.array(bottom_lip[0]))

    # Calculate the horizontal distance
    left_corner = landmarks[78]  # Left corner of the mouth
    right_corner = landmarks[308]  # Right corner of the mouth
    horizontal_distance = np.linalg.norm(np.array(left_corner) - np.array(right_corner))

    # MAR calculation
    mar = vertical_distance / horizontal_distance
    return mar

# Thresholds
YAWN_THRESHOLD = 0.6  # Threshold for detecting a yawn
CLOSE_THRESHOLD = 0.4  # Threshold for detecting mouth close after a yawn
COOLDOWN_PERIOD = 20  # Number of frames to wait before allowing another yawn count

# Start the webcam feed
cap = cv2.VideoCapture(0)

yawn_count = 0
mouth_open = False  # Track if the mouth was open in the previous frame
cooldown = 0  # Cooldown timer to prevent multiple yawn counts

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to detect face landmarks
    result = face_mesh.process(rgb_frame)

    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            landmarks = []
            for lm in face_landmarks.landmark:
                landmarks.append([int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])])

            # Calculate the MAR
            mar = calculate_mar(landmarks)

            # Check if MAR exceeds the yawn threshold (mouth is open)
            if mar > YAWN_THRESHOLD:
                if not mouth_open and cooldown == 0:
                    mouth_open = True
            # Check if MAR is below the close threshold (mouth is closed)
            elif mar < CLOSE_THRESHOLD:
                if mouth_open:
                    yawn_count += 1
                    mouth_open = False
                    cooldown = COOLDOWN_PERIOD  # Start cooldown period
                    cv2.putText(frame, "Yawning!", (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

            # Decrement the cooldown timer if it's active
            if cooldown > 0:
                cooldown -= 1

            # Draw only the mouth landmarks
            mouth_landmarks = [13, 14, 78, 308]
            for idx in mouth_landmarks:
                x, y = landmarks[idx]
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

    # Display yawn count
    cv2.putText(frame, f"Yawn Count: {yawn_count}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow('Yawn Detection', frame)

    # Press 'q' to quit
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

C:\Users\DELL\intern_proj\NEWN\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


# Blink, yawn and head position with commands

In [ ]:
# commands played only once no exceptions are displayed
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
import requests
from datetime import datetime
import time
import pyttsx3
import threading

# Initialize Mediapipe components
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define constants for EAR, MAR, and thresholds
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
MOUTH_AR_THRESH = 0.6  # Yawn threshold
MOUTH_AR_CLOSE_THRESH = 0.4  # Mouth close threshold
MOUTH_AR_CONSEC_FRAMES = 15
DROWSINESS_TIME_THRESHOLD = 5  # 5 seconds for drowsiness detection
HEAD_POSITION_TIME_THRESHOLD = 3  # 10 seconds for head position alert
COOLDOWN_PERIOD = 20  # Cooldown period for yawn counting

# Timer and yawn count settings
YAWN_LIMIT = 3  # Limit for yawn count to trigger fatigue alert
TIMER_DURATION = 40  # Timer duration in seconds

# Indices for landmarks
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
MOUTH = [78, 95, 88, 191, 80, 82, 13, 311, 308, 402, 317, 14]
JAWLINE = list(range(0, 17))  # Jawline (from 0 to 16)

# Initialize counters and time tracking
blink_count = 0
yawn_count = 0
blink_frames = 0
yawn_frames = 0
cooldown = 0  # Cooldown timer to prevent multiple yawn counts
mouth_open = False  # Track if the mouth was open in the previous frame
position = 'center'
eyes_closed_time = None
drowsy_command_played = False
head_position_start_time = None
look_at_road_command_played = False

# Timer variables for yawn detection
timer_start = time.time()  # To track the 30-second interval

# Fetch location using IP-based service
def get_current_location():
    ip_info_url = "http://ipinfo.io/json"
    response = requests.get(ip_info_url)
    data = response.json()
    
    city = data.get('city', 'Unknown City')
    region = data.get('region', 'Unknown Region')
    country = data.get('country', 'Unknown Country')
    
    location = f"{city}, {region}, {country}"
    
    return location

location_text = get_current_location()

# Define EAR and MAR calculation functions
def calculate_ear(eye_landmarks):
    vert1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vert2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    horz = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (vert1 + vert2) / (2.0 * horz)
    return ear

def calculate_mar(landmarks):
    top_lip = landmarks[13:15]  # Top lip points
    bottom_lip = landmarks[14:16]  # Bottom lip points

    # Calculate the vertical distance
    vertical_distance = np.linalg.norm(np.array(top_lip[0]) - np.array(bottom_lip[0]))

    # Calculate the horizontal distance
    left_corner = landmarks[78]  # Left corner of the mouth
    right_corner = landmarks[308]  # Right corner of the mouth
    horizontal_distance = np.linalg.norm(np.array(left_corner) - np.array(right_corner))

    # MAR calculation
    mar = vertical_distance / horizontal_distance
    return mar

def get_head_position(jawline, frame_width, frame_height):
    jawline_x = jawline[:, 0]
    jawline_y = jawline[:, 1]
    
    jaw_center_x = np.mean(jawline_x)
    jaw_center_y = np.mean(jawline_y)
    
    center_x = frame_width / 2
    center_y = frame_height / 2
    
    horizontal_threshold = 60  # Threshold for detecting horizontal movement
    vertical_threshold = 50    # Threshold for detecting vertical movement
    
    if jaw_center_x < center_x - horizontal_threshold:
        return 'left'
    elif jaw_center_x > center_x + horizontal_threshold:
        return 'right'
    elif jaw_center_y < center_y - vertical_threshold:
        return 'up'
    elif jaw_center_y > center_y + vertical_threshold:
        return 'down'
    else:
        return 'center'

# Function to calculate pupil center
def get_pupil_center(eye_landmarks):
    x_coords = [pt[0] for pt in eye_landmarks]
    y_coords = [pt[1] for pt in eye_landmarks]
    center_x = int(sum(x_coords) / len(x_coords))
    center_y = int(sum(y_coords) / len(y_coords))
    return (center_x, center_y)

# Function to play voice command
def play_voice_command(message):
    engine.say(message)
    engine.runAndWait()

# Function to run voice command in a separate thread
def thread_play_voice_command(message):
    t = threading.Thread(target=play_voice_command, args=(message,))
    t.start()

# Open video capture with 720p resolution
cap = cv2.VideoCapture(0)
width = 1280
height = 720
fps = 30

cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FPS, fps)

cv2.namedWindow('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Timer variables for yawn detection
timer_start = time.time()  # To track the 30-second interval
yawn_timer_start = time.time()  # To track the 30-second window for yawn counts

# Initialize a flag to track if the fatigue command has been played
fatigue_command_played = False

# ... (other parts of the code remain unchanged)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = np.array([(lm.x * frame_width, lm.y * frame_height) for lm in face_landmarks.landmark])

            # Extract landmarks
            left_eye = landmarks[LEFT_EYE]
            right_eye = landmarks[RIGHT_EYE]
            mouth = landmarks[MOUTH]
            jawline = landmarks[JAWLINE]

            # Calculate EAR and MAR
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = calculate_mar(landmarks)

            # Blink detection logic
            if ear < EYE_AR_THRESH:
                blink_frames += 1

                # Start counting the duration of eye closure
                if eyes_closed_time is None:
                    eyes_closed_time = time.time()

                # Check if eyes are closed for more than the threshold
                if time.time() - eyes_closed_time >= DROWSINESS_TIME_THRESHOLD:
                    # Continuously play the voice command while eyes are closed
                    if not drowsy_command_played:
                        thread_play_voice_command("You are feeling drowsy! Stay alert")
                        drowsy_command_played = True

            else:
                if blink_frames >= EYE_AR_CONSEC_FRAMES:
                    blink_count += 1

                blink_frames = 0
                eyes_closed_time = None
                drowsy_command_played = False

            # Yawn detection logic
            if mar > MOUTH_AR_THRESH:
                if not mouth_open and cooldown == 0:
                    mouth_open = True
            elif mar < MOUTH_AR_CLOSE_THRESH:
                if mouth_open:
                    yawn_count += 1
                    mouth_open = False
                    cooldown = COOLDOWN_PERIOD  # Start cooldown period

            # Decrease the cooldown timer
            if cooldown > 0:
                cooldown -= 1

            # Check if yawn count exceeds the threshold
            if yawn_count >= YAWN_LIMIT:
                # Play the voice command immediately
                thread_play_voice_command("You look fatigued! Take rest")
                
                cv2.putText(frame, f'Fatigue state', (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Reset the yawn count, timer, and fatigue command flag immediately
                yawn_count = 0
                yawn_timer_start = time.time()
                fatigue_command_played = False  # Reset the fatigue command flag for future use

            # Timer logic for yawn count reset, independent of the yawn threshold
            if time.time() - yawn_timer_start >= TIMER_DURATION:
                yawn_timer_start = time.time()
                yawn_count = 0
                fatigue_command_played = False  # Ensure the fatigue command can be played again


            # Calculate head position
            position = get_head_position(jawline, frame_width, frame_height)

            # Check head position for voice command
            if position != 'center':
                if not look_at_road_command_played:  # Check if the command hasn't been played yet
                    if head_position_start_time is None:
                        head_position_start_time = time.time()
            
                    if time.time() - head_position_start_time >= HEAD_POSITION_TIME_THRESHOLD:
                        thread_play_voice_command("Look at the road")
                        look_at_road_command_played = True  # Set flag to True to avoid replaying
            
                        cv2.putText(frame, f'Cognitive distraction', (10, 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                # Reset flag and time when head returns to center
                head_position_start_time = None
                look_at_road_command_played = False


            # Calculate pupil centers
            pupil_left = get_pupil_center(left_eye)
            pupil_right = get_pupil_center(right_eye)

    # Display blink and yawn counts
    cv2.putText(frame, f'Blinks: {blink_count}', (10, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Yawns: {yawn_count}', (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display head position
    cv2.putText(frame, f'Head Position: {position}', (10, 110),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display pupil markers
    cv2.circle(frame, pupil_left, 3, (0, 255, 0), -1)
    cv2.circle(frame, pupil_right, 3, (0, 255, 0), -1)

    # Display date, time, and location
    now = datetime.now()
    date_str = now.strftime('%d-%m-%Y')
    time_str = now.strftime('%H:%M:%S')

    cv2.putText(frame, f'{date_str}', (frame_width - 200, frame_height - 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'{time_str}', (frame_width - 200, frame_height - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'{location_text}', (10, frame_height - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Face Landmarks Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# commands with text in display
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
import requests
from datetime import datetime
import time
import pyttsx3
import threading

# Initialize Mediapipe components
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define constants for EAR, MAR, and thresholds
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
MOUTH_AR_THRESH = 0.6  # Yawn threshold
MOUTH_AR_CLOSE_THRESH = 0.4  # Mouth close threshold
MOUTH_AR_CONSEC_FRAMES = 15
DROWSINESS_TIME_THRESHOLD = 5  # 5 seconds for drowsiness detection
HEAD_POSITION_TIME_THRESHOLD = 3  # 10 seconds for head position alert
COOLDOWN_PERIOD = 20  # Cooldown period for yawn counting

# Timer and yawn count settings
YAWN_LIMIT = 3  # Limit for yawn count to trigger fatigue alert
TIMER_DURATION = 40  # Timer duration in seconds

# Indices for landmarks
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
MOUTH = [78, 95, 88, 191, 80, 82, 13, 311, 308, 402, 317, 14]
JAWLINE = list(range(0, 17))  # Jawline (from 0 to 16)

# Initialize counters and time tracking
blink_count = 0
yawn_count = 0
blink_frames = 0
yawn_frames = 0
cooldown = 0  # Cooldown timer to prevent multiple yawn counts
mouth_open = False  # Track if the mouth was open in the previous frame
position = 'center'
eyes_closed_time = None
drowsy_command_played = False
head_position_start_time = None
look_at_road_command_played = False

# Timer variables for yawn detection
timer_start = time.time()  # To track the 30-second interval

# Fetch location using IP-based service
def get_current_location():
    ip_info_url = "http://ipinfo.io/json"
    response = requests.get(ip_info_url)
    data = response.json()
    
    city = data.get('city', 'Unknown City')
    region = data.get('region', 'Unknown Region')
    country = data.get('country', 'Unknown Country')
    
    location = f"{city}, {region}, {country}"
    
    return location

location_text = get_current_location()

# Define EAR and MAR calculation functions
def calculate_ear(eye_landmarks):
    vert1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vert2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    horz = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (vert1 + vert2) / (2.0 * horz)
    return ear

def calculate_mar(landmarks):
    top_lip = landmarks[13:15]  # Top lip points
    bottom_lip = landmarks[14:16]  # Bottom lip points

    # Calculate the vertical distance
    vertical_distance = np.linalg.norm(np.array(top_lip[0]) - np.array(bottom_lip[0]))

    # Calculate the horizontal distance
    left_corner = landmarks[78]  # Left corner of the mouth
    right_corner = landmarks[308]  # Right corner of the mouth
    horizontal_distance = np.linalg.norm(np.array(left_corner) - np.array(right_corner))

    # MAR calculation
    mar = vertical_distance / horizontal_distance
    return mar

def get_head_position(jawline, frame_width, frame_height):
    jawline_x = jawline[:, 0]
    jawline_y = jawline[:, 1]
    
    jaw_center_x = np.mean(jawline_x)
    jaw_center_y = np.mean(jawline_y)
    
    center_x = frame_width / 2
    center_y = frame_height / 2
    
    horizontal_threshold = 60  # Threshold for detecting horizontal movement
    vertical_threshold = 50    # Threshold for detecting vertical movement
    
    if jaw_center_x < center_x - horizontal_threshold:
        return 'left'
    elif jaw_center_x > center_x + horizontal_threshold:
        return 'right'
    elif jaw_center_y < center_y - vertical_threshold:
        return 'up'
    elif jaw_center_y > center_y + vertical_threshold:
        return 'down'
    else:
        return 'center'

# Function to calculate pupil center
def get_pupil_center(eye_landmarks):
    x_coords = [pt[0] for pt in eye_landmarks]
    y_coords = [pt[1] for pt in eye_landmarks]
    center_x = int(sum(x_coords) / len(x_coords))
    center_y = int(sum(y_coords) / len(y_coords))
    return (center_x, center_y)

# Function to play voice command
def play_voice_command(message):
    engine.say(message)
    engine.runAndWait()

# Function to run voice command in a separate thread
def thread_play_voice_command(message):
    t = threading.Thread(target=play_voice_command, args=(message,))
    t.start()

# Open video capture with 720p resolution
cap = cv2.VideoCapture(0)
width = 1280
height = 720
fps = 30

cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FPS, fps)

cv2.namedWindow('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Timer variables for yawn detection
timer_start = time.time()  # To track the 30-second interval
yawn_timer_start = time.time()  # To track the 30-second window for yawn counts

# Initialize a flag to track if the fatigue command has been played
fatigue_command_played = False

# ... (other parts of the code remain unchanged)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = np.array([(lm.x * frame_width, lm.y * frame_height) for lm in face_landmarks.landmark])

            # Extract landmarks
            left_eye = landmarks[LEFT_EYE]
            right_eye = landmarks[RIGHT_EYE]
            mouth = landmarks[MOUTH]
            jawline = landmarks[JAWLINE]

            # Calculate EAR and MAR
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = calculate_mar(landmarks)

            # Blink detection logic
            if ear < EYE_AR_THRESH:
                blink_frames += 1

                # Start counting the duration of eye closure
                if eyes_closed_time is None:
                    eyes_closed_time = time.time()

                # Check if eyes are closed for more than the threshold
                if time.time() - eyes_closed_time >= DROWSINESS_TIME_THRESHOLD:
                    # Continuously play the voice command while eyes are closed
                    if not drowsy_command_played:
                        thread_play_voice_command("You are feeling drowsy! Stay alert")
                        drowsy_command_played = True
                        
                    else:
                        # Re-play the command if it's already playing
                        thread_play_voice_command("You are feeling drowsy! Stay alert")
                        cv2.putText(frame, f'Drowsy state', (10, 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            else:
                if blink_frames >= EYE_AR_CONSEC_FRAMES:
                    blink_count += 1

                blink_frames = 0
                eyes_closed_time = None
                drowsy_command_played = False

            # Yawn detection logic
            if mar > MOUTH_AR_THRESH:
                if not mouth_open and cooldown == 0:
                    mouth_open = True
            elif mar < MOUTH_AR_CLOSE_THRESH:
                if mouth_open:
                    yawn_count += 1
                    mouth_open = False
                    cooldown = COOLDOWN_PERIOD  # Start cooldown period

            # Decrease the cooldown timer
            if cooldown > 0:
                cooldown -= 1

            # Check if yawn count exceeds the threshold
            if yawn_count >= YAWN_LIMIT:
                # Play the voice command immediately
                thread_play_voice_command("You look fatigued! Take rest")
                
                cv2.putText(frame, f'Fatigue state', (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Reset the yawn count, timer, and fatigue command flag immediately
                yawn_count = 0
                yawn_timer_start = time.time()
                fatigue_command_played = False  # Reset the fatigue command flag for future use

            # Timer logic for yawn count reset, independent of the yawn threshold
            if time.time() - yawn_timer_start >= TIMER_DURATION:
                yawn_timer_start = time.time()
                yawn_count = 0
                fatigue_command_played = False  # Ensure the fatigue command can be played again


            # Calculate head position
            position = get_head_position(jawline, frame_width, frame_height)

            # Check head position for voice command
            if position != 'center':
                if head_position_start_time is None:
                    head_position_start_time = time.time()

                if time.time() - head_position_start_time >= HEAD_POSITION_TIME_THRESHOLD:
                    thread_play_voice_command("Look at the road")
                    look_at_road_command_played = True
                    
                    cv2.putText(frame, f'Cognitive distraction', (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            else:
                head_position_start_time = None
                look_at_road_command_played = False

            # Calculate pupil centers
            pupil_left = get_pupil_center(left_eye)
            pupil_right = get_pupil_center(right_eye)

    # Display blink and yawn counts
    cv2.putText(frame, f'Blinks: {blink_count}', (10, 30),
               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Yawns: {yawn_count}', (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display head position
    cv2.putText(frame, f'Head Position: {position}', (10, 110),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display pupil markers
    cv2.circle(frame, pupil_left, 3, (0, 255, 0), -1)
    cv2.circle(frame, pupil_right, 3, (0, 255, 0), -1)

    # Display date, time, and location
    now = datetime.now()
    date_str = now.strftime('%d-%m-%Y')
    time_str = now.strftime('%H:%M:%S')

    cv2.putText(frame, f'{date_str}', (frame_width - 200, frame_height - 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'{time_str}', (frame_width - 200, frame_height - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'{location_text}', (10, frame_height - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Face Landmarks Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# Perfect code for yawn blink and head position with command
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
import requests
from datetime import datetime
import time
import pyttsx3
import threading

# Initialize Mediapipe components
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Define constants for EAR, MAR, and thresholds
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
MOUTH_AR_THRESH = 0.6  # Yawn threshold
MOUTH_AR_CLOSE_THRESH = 0.4  # Mouth close threshold
MOUTH_AR_CONSEC_FRAMES = 15
DROWSINESS_TIME_THRESHOLD = 5  # 5 seconds for drowsiness detection
HEAD_POSITION_TIME_THRESHOLD = 3  # 10 seconds for head position alert
COOLDOWN_PERIOD = 20  # Cooldown period for yawn counting

# Timer and yawn count settings
YAWN_LIMIT = 3  # Limit for yawn count to trigger fatigue alert
TIMER_DURATION = 40  # Timer duration in seconds

# Indices for landmarks
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
MOUTH = [78, 95, 88, 191, 80, 82, 13, 311, 308, 402, 317, 14]
JAWLINE = list(range(0, 17))  # Jawline (from 0 to 16)

# Initialize counters and time tracking
blink_count = 0
yawn_count = 0
blink_frames = 0
yawn_frames = 0
cooldown = 0  # Cooldown timer to prevent multiple yawn counts
mouth_open = False  # Track if the mouth was open in the previous frame
position = 'center'
eyes_closed_time = None
drowsy_command_played = False
head_position_start_time = None
look_at_road_command_played = False

# Timer variables for yawn detection
timer_start = time.time()  # To track the 30-second interval

# Fetch location using IP-based service
def get_current_location():
    ip_info_url = "http://ipinfo.io/json"
    response = requests.get(ip_info_url)
    data = response.json()
    
    city = data.get('city', 'Unknown City')
    region = data.get('region', 'Unknown Region')
    country = data.get('country', 'Unknown Country')
    
    location = f"{city}, {region}, {country}"
    
    return location

location_text = get_current_location()

# Define EAR and MAR calculation functions
def calculate_ear(eye_landmarks):
    vert1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vert2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    horz = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (vert1 + vert2) / (2.0 * horz)
    return ear

def calculate_mar(landmarks):
    top_lip = landmarks[13:15]  # Top lip points
    bottom_lip = landmarks[14:16]  # Bottom lip points

    # Calculate the vertical distance
    vertical_distance = np.linalg.norm(np.array(top_lip[0]) - np.array(bottom_lip[0]))

    # Calculate the horizontal distance
    left_corner = landmarks[78]  # Left corner of the mouth
    right_corner = landmarks[308]  # Right corner of the mouth
    horizontal_distance = np.linalg.norm(np.array(left_corner) - np.array(right_corner))

    # MAR calculation
    mar = vertical_distance / horizontal_distance
    return mar

def get_head_position(jawline, frame_width, frame_height):
    jawline_x = jawline[:, 0]
    jawline_y = jawline[:, 1]
    
    jaw_center_x = np.mean(jawline_x)
    jaw_center_y = np.mean(jawline_y)
    
    center_x = frame_width / 2
    center_y = frame_height / 2
    
    horizontal_threshold = 60  # Threshold for detecting horizontal movement
    vertical_threshold = 50    # Threshold for detecting vertical movement
    
    if jaw_center_x < center_x - horizontal_threshold:
        return 'left'
    elif jaw_center_x > center_x + horizontal_threshold:
        return 'right'
    elif jaw_center_y < center_y - vertical_threshold:
        return 'up'
    elif jaw_center_y > center_y + vertical_threshold:
        return 'down'
    else:
        return 'center'

# Function to calculate pupil center
def get_pupil_center(eye_landmarks):
    x_coords = [pt[0] for pt in eye_landmarks]
    y_coords = [pt[1] for pt in eye_landmarks]
    center_x = int(sum(x_coords) / len(x_coords))
    center_y = int(sum(y_coords) / len(y_coords))
    return (center_x, center_y)

# Function to play voice command
def play_voice_command(message):
    engine.say(message)
    engine.runAndWait()

# Function to run voice command in a separate thread
def thread_play_voice_command(message):
    t = threading.Thread(target=play_voice_command, args=(message,))
    t.start()

# Open video capture with 720p resolution
cap = cv2.VideoCapture(0)
width = 1280
height = 720
fps = 30

cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FPS, fps)

cv2.namedWindow('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Timer variables for yawn detection
timer_start = time.time()  # To track the 30-second interval
yawn_timer_start = time.time()  # To track the 30-second window for yawn counts

# Initialize a flag to track if the fatigue command has been played
fatigue_command_played = False

# ... (other parts of the code remain unchanged)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = np.array([(lm.x * frame_width, lm.y * frame_height) for lm in face_landmarks.landmark])

            # Extract landmarks
            left_eye = landmarks[LEFT_EYE]
            right_eye = landmarks[RIGHT_EYE]
            mouth = landmarks[MOUTH]
            jawline = landmarks[JAWLINE]

            # Calculate EAR and MAR
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = calculate_mar(landmarks)

            # Blink detection logic
            if ear < EYE_AR_THRESH:
                blink_frames += 1

                # Start counting the duration of eye closure
                if eyes_closed_time is None:
                    eyes_closed_time = time.time()

                # Check if eyes are closed for more than the threshold
                if time.time() - eyes_closed_time >= DROWSINESS_TIME_THRESHOLD:
                    # Continuously play the voice command while eyes are closed
                    if not drowsy_command_played:
                        thread_play_voice_command("You are feeling drowsy! Stay alert")
                        drowsy_command_played = True
                        
                    else:
                        # Re-play the command if it's already playing
                        thread_play_voice_command("You are feeling drowsy! Stay alert")
                        cv2.putText(frame, f'Drowsy state', (10, 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            else:
                if blink_frames >= EYE_AR_CONSEC_FRAMES:
                    blink_count += 1

                blink_frames = 0
                eyes_closed_time = None
                drowsy_command_played = False

            # Yawn detection logic
            if mar > MOUTH_AR_THRESH:
                if not mouth_open and cooldown == 0:
                    mouth_open = True
            elif mar < MOUTH_AR_CLOSE_THRESH:
                if mouth_open:
                    yawn_count += 1
                    mouth_open = False
                    cooldown = COOLDOWN_PERIOD  # Start cooldown period

            # Decrease the cooldown timer
            if cooldown > 0:
                cooldown -= 1

            # Check if yawn count exceeds the threshold
            if yawn_count >= YAWN_LIMIT:
                # Play the voice command immediately
                thread_play_voice_command("You look fatigued! Take rest")

                # Reset the yawn count, timer, and fatigue command flag immediately
                yawn_count = 0
                yawn_timer_start = time.time()
                fatigue_command_played = False  # Reset the fatigue command flag for future use

            # Timer logic for yawn count reset, independent of the yawn threshold
            if time.time() - yawn_timer_start >= TIMER_DURATION:
                yawn_timer_start = time.time()
                yawn_count = 0
                fatigue_command_played = False  # Ensure the fatigue command can be played again


            # Calculate head position
            position = get_head_position(jawline, frame_width, frame_height)

            # Check head position for voice command
            if position != 'center':
                if head_position_start_time is None:
                    head_position_start_time = time.time()

                if time.time() - head_position_start_time >= HEAD_POSITION_TIME_THRESHOLD:
                    thread_play_voice_command("Look at the road")
                    look_at_road_command_played = True

            else:
                head_position_start_time = None
                look_at_road_command_played = False

            # Calculate pupil centers
            pupil_left = get_pupil_center(left_eye)
            pupil_right = get_pupil_center(right_eye)

    # Display blink and yawn counts
    cv2.putText(frame, f'Blinks: {blink_count}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Yawns: {yawn_count}', (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display head position
    cv2.putText(frame, f'Head Position: {position}', (10, 110),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display pupil markers
    #cv2.circle(frame, pupil_left, 3, (0, 255, 0), -1)
    #cv2.circle(frame, pupil_right, 3, (0, 255, 0), -1)

    # Display date, time, and location
    now = datetime.now()
    date_str = now.strftime('%d-%m-%Y')
    time_str = now.strftime('%H:%M:%S')

    cv2.putText(frame, f'{date_str}', (frame_width - 200, frame_height - 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'{time_str}', (frame_width - 200, frame_height - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'{location_text}', (10, frame_height - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Face Landmarks Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [5]:
cap.release()
cv2.destroyAllWindows()

# Email

In [21]:
# only eye with mail
import cv2
import numpy as np
import mediapipe as mp
import time
import pyttsx3
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Initialize Mediapipe components
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Email configuration
SENDER_EMAIL = "karthiiiksk@gmail.com"  # Change this to your email
SENDER_PASSWORD = "bpql sxcu hpkt mqdp"
RECIPIENT_EMAIL = "srinivasakarthik4522@gmail.com"  # Change this to the recipient's email

# Define constants for EAR and thresholds
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3  # Number of consecutive frames to consider a blink
DROWSINESS_TIME_THRESHOLD = 5  # 5 seconds for voice alert
EMAIL_ALERT_THRESHOLD = 15  # 15 seconds for email alert

# Email sending function
def send_email():
    try:
        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = RECIPIENT_EMAIL
        msg['Subject'] = "Alert: Person might have fainted!"
        
        body = "The person has closed their eyes for more than 15 seconds and may have fainted."
        msg.attach(MIMEText(body, 'plain'))
        
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(SENDER_EMAIL, SENDER_PASSWORD)
        
        server.sendmail(SENDER_EMAIL, RECIPIENT_EMAIL, msg.as_string())
        server.quit()
        print("Alert email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye_landmarks):
    A = np.linalg.norm(eye_landmarks[1] - eye_landmarks[5])
    B = np.linalg.norm(eye_landmarks[2] - eye_landmarks[4])
    C = np.linalg.norm(eye_landmarks[0] - eye_landmarks[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Function to play voice command
def play_voice_command(message):
    engine.say(message)
    engine.runAndWait()

# Open video capture with 720p resolution
cap = cv2.VideoCapture(0)
width = 1280
height = 720
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# Initialize counters and time tracking
blink_frames = 0
blink_count = 0
eyes_closed_time = None
voice_alert_played = False
email_alert_sent = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = np.array([(lm.x * frame_width, lm.y * frame_height) for lm in face_landmarks.landmark])

            # Extract left and right eye landmarks (example indices for eyes)
            left_eye = landmarks[[33, 160, 158, 133, 153, 144]]
            right_eye = landmarks[[362, 385, 387, 263, 373, 380]]

            # Calculate EAR for both eyes
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0

            # Blink detection logic
            if ear < EYE_AR_THRESH:
                blink_frames += 1
                # Start timing how long the eyes are closed
                if eyes_closed_time is None:
                    eyes_closed_time = time.time()

                # Voice alert after 5 seconds of closed eyes
                if time.time() - eyes_closed_time >= DROWSINESS_TIME_THRESHOLD and not voice_alert_played:
                    play_voice_command("You are feeling drowsy! Stay alert.")
                    voice_alert_played = True

                # Email alert after 15 seconds of closed eyes
                if time.time() - eyes_closed_time >= EMAIL_ALERT_THRESHOLD and not email_alert_sent:
                    send_email()
                    email_alert_sent = True
            else:
                if blink_frames >= EYE_AR_CONSEC_FRAMES:
                    blink_count += 1
                blink_frames = 0
                eyes_closed_time = None
                voice_alert_played = False
                email_alert_sent = False

    # Display EAR and blink count
    cv2.putText(frame, f'EAR: {ear:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Blinks: {blink_count}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow('Eye Drowsiness and Blink Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Blink Yawn Head Mail

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
import requests
from datetime import datetime
import time
import pyttsx3
import threading
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Initialize Mediapipe components
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Email configuration
SENDER_EMAIL = "karthiiiksk@gmail.com"  # Change this to your email
SENDER_PASSWORD = "bpql sxcu hpkt mqdp"  
RECIPIENT_EMAIL = "srinivasakarthik4522@gmail.com"  # Change this to the recipient's email

# Define constants for EAR, MAR, and thresholds
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
MOUTH_AR_THRESH = 0.6  # Yawn threshold
MOUTH_AR_CLOSE_THRESH = 0.4  # Mouth close threshold
MOUTH_AR_CONSEC_FRAMES = 15
DROWSINESS_TIME_THRESHOLD = 5  # 5 seconds for voice alert
EMAIL_ALERT_THRESHOLD = 15  # 15 seconds for email alert
HEAD_POSITION_TIME_THRESHOLD = 3  # 10 seconds for head position alert
COOLDOWN_PERIOD = 20  # Cooldown period for yawn counting

# Timer and yawn count settings
YAWN_LIMIT = 3  # Limit for yawn count to trigger fatigue alert
TIMER_DURATION = 40  # Timer duration in seconds

# Indices for landmarks
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
MOUTH = [78, 95, 88, 191, 80, 82, 13, 311, 308, 402, 317, 14]
JAWLINE = list(range(0, 17))  # Jawline (from 0 to 16)

# Initialize counters and time tracking
blink_count = 0
yawn_count = 0
blink_frames = 0
yawn_frames = 0
cooldown = 0  # Cooldown timer to prevent multiple yawn counts
mouth_open = False  # Track if the mouth was open in the previous frame
position = 'center'
eyes_closed_time = None
drowsy_command_played = False
head_position_start_time = None
look_at_road_command_played = False
voice_alert_played = False
email_alert_sent = False

# Timer variables for yawn detection
timer_start = time.time()  # To track the 30-second interval

# Fetch location using IP-based service
def get_current_location():
    ip_info_url = "http://ipinfo.io/json"
    response = requests.get(ip_info_url)
    data = response.json()
    
    city = data.get('city', 'Unknown City')
    region = data.get('region', 'Unknown Region')
    country = data.get('country', 'Unknown Country')
    
    location = f"{city}, {region}, {country}"
    
    return location

location_text = get_current_location()

def send_email():
    try:
        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = RECIPIENT_EMAIL
        msg['Subject'] = "Alert: Person might have fainted!"
        
        body = "The person has closed their eyes for more than 15 seconds and may have fainted."
        msg.attach(MIMEText(body, 'plain'))
        
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(SENDER_EMAIL, SENDER_PASSWORD)
        
        server.sendmail(SENDER_EMAIL, RECIPIENT_EMAIL, msg.as_string())
        server.quit()
        print("Alert email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")
        
# Define EAR and MAR calculation functions
def calculate_ear(eye_landmarks):
    vert1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vert2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    horz = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (vert1 + vert2) / (2.0 * horz)
    return ear

def calculate_mar(landmarks):
    top_lip = landmarks[13:15]  # Top lip points
    bottom_lip = landmarks[14:16]  # Bottom lip points

    # Calculate the vertical distance
    vertical_distance = np.linalg.norm(np.array(top_lip[0]) - np.array(bottom_lip[0]))

    # Calculate the horizontal distance
    left_corner = landmarks[78]  # Left corner of the mouth
    right_corner = landmarks[308]  # Right corner of the mouth
    horizontal_distance = np.linalg.norm(np.array(left_corner) - np.array(right_corner))

    # MAR calculation
    mar = vertical_distance / horizontal_distance
    return mar

def get_head_position(jawline, frame_width, frame_height):
    jawline_x = jawline[:, 0]
    jawline_y = jawline[:, 1]
    
    jaw_center_x = np.mean(jawline_x)
    jaw_center_y = np.mean(jawline_y)
    
    center_x = frame_width / 2
    center_y = frame_height / 2
    
    horizontal_threshold = 60  # Threshold for detecting horizontal movement
    vertical_threshold = 50    # Threshold for detecting vertical movement
    
    if jaw_center_x < center_x - horizontal_threshold:
        return 'left'
    elif jaw_center_x > center_x + horizontal_threshold:
        return 'right'
    elif jaw_center_y < center_y - vertical_threshold:
        return 'up'
    elif jaw_center_y > center_y + vertical_threshold:
        return 'down'
    else:
        return 'center'

# Function to calculate pupil center
def get_pupil_center(eye_landmarks):
    x_coords = [pt[0] for pt in eye_landmarks]
    y_coords = [pt[1] for pt in eye_landmarks]
    center_x = int(sum(x_coords) / len(x_coords))
    center_y = int(sum(y_coords) / len(y_coords))
    return (center_x, center_y)

# Function to play voice command
def play_voice_command(message):
    engine.say(message)
    engine.runAndWait()

# Function to run voice command in a separate thread
def thread_play_voice_command(message):
    t = threading.Thread(target=play_voice_command, args=(message,))
    t.start()

# Open video capture with 720p resolution
cap = cv2.VideoCapture(0)
width = 1280
height = 720
fps = 30

cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FPS, fps)

cv2.namedWindow('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Timer variables for yawn detection
timer_start = time.time()  # To track the 30-second interval
yawn_timer_start = time.time()  # To track the 30-second window for yawn counts

# Initialize a flag to track if the fatigue command has been played
fatigue_command_played = False

# ... (other parts of the code remain unchanged)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = np.array([(lm.x * frame_width, lm.y * frame_height) for lm in face_landmarks.landmark])

            # Extract landmarks
            left_eye = landmarks[LEFT_EYE]
            right_eye = landmarks[RIGHT_EYE]
            mouth = landmarks[MOUTH]
            jawline = landmarks[JAWLINE]

            # Calculate EAR and MAR
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = calculate_mar(landmarks)

            # Blink detection logic
            if ear < EYE_AR_THRESH:
                blink_frames += 1

                # Start counting the duration of eye closure
                if eyes_closed_time is None:
                    eyes_closed_time = time.time()

                if time.time() - eyes_closed_time >= DROWSINESS_TIME_THRESHOLD:
                    # Continuously play the voice command while eyes are closed
                    if not drowsy_command_played:
                        thread_play_voice_command("You are feeling drowsy! Stay alert")
                        drowsy_command_played = True
                    else:
                        # Re-play the command if it's already playing
                        thread_play_voice_command("You are feeling drowsy! Stay alert")
                        cv2.putText(frame, f'Drowsy state', (10, 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        
                        if time.time() - eyes_closed_time >= EMAIL_ALERT_THRESHOLD and not email_alert_sent:
                            print("Attempting to send email...")
                            send_email()
                            email_alert_sent = True

            else:
                if blink_frames >= EYE_AR_CONSEC_FRAMES:
                    blink_count += 1

                blink_frames = 0
                eyes_closed_time = None
                drowsy_command_played = False
                email_alert_sent = False

            # Yawn detection logic
            if mar > MOUTH_AR_THRESH:
                if not mouth_open and cooldown == 0:
                    mouth_open = True
            elif mar < MOUTH_AR_CLOSE_THRESH:
                if mouth_open:
                    yawn_count += 1
                    mouth_open = False
                    cooldown = COOLDOWN_PERIOD  # Start cooldown period

            # Decrease the cooldown timer
            if cooldown > 0:
                cooldown -= 1

            # Check if yawn count exceeds the threshold
            if yawn_count >= YAWN_LIMIT:
                # Play the voice command immediately
                thread_play_voice_command("You look fatigued! Take rest")

                # Reset the yawn count, timer, and fatigue command flag immediately
                yawn_count = 0
                yawn_timer_start = time.time()
                fatigue_command_played = False  # Reset the fatigue command flag for future use

            # Timer logic for yawn count reset, independent of the yawn threshold
            if time.time() - yawn_timer_start >= TIMER_DURATION:
                yawn_timer_start = time.time()
                yawn_count = 0
                fatigue_command_played = False  # Ensure the fatigue command can be played again


            # Calculate head position
            position = get_head_position(jawline, frame_width, frame_height)

            # Check head position for voice command
            if position != 'center':
                if head_position_start_time is None:
                    head_position_start_time = time.time()

                if time.time() - head_position_start_time >= HEAD_POSITION_TIME_THRESHOLD:
                    thread_play_voice_command("Look at the road")
                    look_at_road_command_played = True

            else:
                head_position_start_time = None
                look_at_road_command_played = False

            # Calculate pupil centers
            pupil_left = get_pupil_center(left_eye)
            pupil_right = get_pupil_center(right_eye)

    # Display blink and yawn counts
    cv2.putText(frame, f'Blinks: {blink_count}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Yawns: {yawn_count}', (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display head position
    cv2.putText(frame, f'Head Position: {position}', (10, 110),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display pupil markers
    cv2.circle(frame, pupil_left, 3, (0, 255, 0), -1)
    cv2.circle(frame, pupil_right, 3, (0, 255, 0), -1)

    # Display date, time, and location
    now = datetime.now()
    date_str = now.strftime('%d-%m-%Y')
    time_str = now.strftime('%H:%M:%S')

    cv2.putText(frame, f'{date_str}', (frame_width - 200, frame_height - 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'{time_str}', (frame_width - 200, frame_height - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'{location_text}', (10, frame_height - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Face Landmarks Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Icon

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.spatial import distance as dist
import requests
from datetime import datetime
import time
import pyttsx3
import threading
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Initialize Mediapipe components
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Email configuration
SENDER_EMAIL = "karthiiiksk@gmail.com"  # Change this to your email
SENDER_PASSWORD = "bpql sxcu hpkt mqdp"  
RECIPIENT_EMAIL = "srinivasakarthik4522@gmail.com"  # Change this to the recipient's email

# Define constants for EAR, MAR, and thresholds
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3
MOUTH_AR_THRESH = 0.6  # Yawn threshold
MOUTH_AR_CLOSE_THRESH = 0.4  # Mouth close threshold
MOUTH_AR_CONSEC_FRAMES = 15
DROWSINESS_TIME_THRESHOLD = 5  # 5 seconds for voice alert
EMAIL_ALERT_THRESHOLD = 15  # 15 seconds for email alert
HEAD_POSITION_TIME_THRESHOLD = 3  # 10 seconds for head position alert
COOLDOWN_PERIOD = 20  # Cooldown period for yawn counting

# Timer and yawn count settings
YAWN_LIMIT = 3  # Limit for yawn count to trigger fatigue alert
TIMER_DURATION = 40  # Timer duration in seconds

# Indices for landmarks
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
MOUTH = [78, 95, 88, 191, 80, 82, 13, 311, 308, 402, 317, 14]
JAWLINE = list(range(0, 17))  # Jawline (from 0 to 16)

# Initialize counters and time tracking
blink_count = 0
yawn_count = 0
blink_frames = 0
yawn_frames = 0
cooldown = 0  # Cooldown timer to prevent multiple yawn counts
mouth_open = False  # Track if the mouth was open in the previous frame
position = 'center'
eyes_closed_time = None
drowsy_command_played = False
head_position_start_time = None
look_at_road_command_played = False
voice_alert_played = False
email_alert_sent = False

# Timer variables for yawn detection
timer_start = time.time()  # To track the 30-second interval

# Fetch location using IP-based service
def get_current_location():
    ip_info_url = "http://ipinfo.io/json"
    response = requests.get(ip_info_url)
    data = response.json()
    
    city = data.get('city', 'Unknown City')
    region = data.get('region', 'Unknown Region')
    country = data.get('country', 'Unknown Country')
    
    location = f"{city}, {region}, {country}"
    
    return location

location_text = get_current_location()

def send_email():
    try:
        msg = MIMEMultipart()
        msg['From'] = SENDER_EMAIL
        msg['To'] = RECIPIENT_EMAIL
        msg['Subject'] = "Alert: Person might have fainted!"
        
        body = "The person has closed their eyes for more than 15 seconds and may have fainted."
        msg.attach(MIMEText(body, 'plain'))
        
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(SENDER_EMAIL, SENDER_PASSWORD)
        
        server.sendmail(SENDER_EMAIL, RECIPIENT_EMAIL, msg.as_string())
        server.quit()
        print("Alert email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")
        
# Define EAR and MAR calculation functions
def calculate_ear(eye_landmarks):
    vert1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vert2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    horz = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (vert1 + vert2) / (2.0 * horz)
    return ear

def calculate_mar(landmarks):
    top_lip = landmarks[13:15]  # Top lip points
    bottom_lip = landmarks[14:16]  # Bottom lip points

    # Calculate the vertical distance
    vertical_distance = np.linalg.norm(np.array(top_lip[0]) - np.array(bottom_lip[0]))

    # Calculate the horizontal distance
    left_corner = landmarks[78]  # Left corner of the mouth
    right_corner = landmarks[308]  # Right corner of the mouth
    horizontal_distance = np.linalg.norm(np.array(left_corner) - np.array(right_corner))

    # MAR calculation
    mar = vertical_distance / horizontal_distance
    return mar

def get_head_position(jawline, frame_width, frame_height):
    jawline_x = jawline[:, 0]
    jawline_y = jawline[:, 1]
    
    jaw_center_x = np.mean(jawline_x)
    jaw_center_y = np.mean(jawline_y)
    
    center_x = frame_width / 2
    center_y = frame_height / 2
    
    horizontal_threshold = 60  # Threshold for detecting horizontal movement
    vertical_threshold = 50    # Threshold for detecting vertical movement
    
    if jaw_center_x < center_x - horizontal_threshold:
        return 'left'
    elif jaw_center_x > center_x + horizontal_threshold:
        return 'right'
    elif jaw_center_y < center_y - vertical_threshold:
        return 'up'
    elif jaw_center_y > center_y + vertical_threshold:
        return 'down'
    else:
        return 'center'

# Function to calculate pupil center
def get_pupil_center(eye_landmarks):
    x_coords = [pt[0] for pt in eye_landmarks]
    y_coords = [pt[1] for pt in eye_landmarks]
    center_x = int(sum(x_coords) / len(x_coords))
    center_y = int(sum(y_coords) / len(y_coords))
    return (center_x, center_y)

# Function to play voice command
def play_voice_command(message):
    engine.say(message)
    engine.runAndWait()

# Function to run voice command in a separate thread
def thread_play_voice_command(message):
    t = threading.Thread(target=play_voice_command, args=(message,))
    t.start()

# Open video capture with 720p resolution
cap = cv2.VideoCapture(0)
width = 1280
height = 720
fps = 30

cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
cap.set(cv2.CAP_PROP_FPS, fps)

cv2.namedWindow('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty('Face Landmarks Detection', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

# Timer variables for yawn detection
timer_start = time.time()  # To track the 30-second interval
yawn_timer_start = time.time()  # To track the 30-second window for yawn counts

# Initialize a flag to track if the fatigue command has been played
fatigue_command_played = False

# ... (other parts of the code remain unchanged)

# Load images for blink, yawn, and head position
drowsy_img = cv2.imread('C:/Users/DELL/Downloads/blink1_icon.png', cv2.IMREAD_UNCHANGED)
fatigue_img = cv2.imread('C:/Users/DELL/Downloads/yawn1_icon.png', cv2.IMREAD_UNCHANGED)
look_straight_img = cv2.imread('C:/Users/DELL/Downloads/head3_icon.png', cv2.IMREAD_UNCHANGED)

def overlay_image_alpha(background, overlay, position):
    x, y = position
    bg_h, bg_w = background.shape[:2]
    h, w = overlay.shape[:2]

    # Ensure the overlay fits within the frame
    if x + w > bg_w:
        x = bg_w - w
    if y + h > bg_h:
        y = bg_h - h
    
    # Extract the alpha channel
    if x < 0 or y < 0 or x + w > bg_w or y + h > bg_h:
        raise ValueError("Overlay image exceeds background dimensions.")

    alpha_overlay = overlay[:, :, 3] / 255.0
    alpha_background = 1.0 - alpha_overlay

    for c in range(0, 3):
        background[y:y+h, x:x+w, c] = (alpha_overlay * overlay[:, :, c] +
                                       alpha_background * background[y:y+h, x:x+w, c])

    return background


# Inside your while loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    # Reset image display flags
    drowsy_img_displayed = False
    fatigue_img_displayed = False
    look_straight_img_displayed = False

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = np.array([(lm.x * frame_width, lm.y * frame_height) for lm in face_landmarks.landmark])

            # Extract landmarks
            left_eye = landmarks[LEFT_EYE]
            right_eye = landmarks[RIGHT_EYE]
            mouth = landmarks[MOUTH]
            jawline = landmarks[JAWLINE]

            # Calculate EAR and MAR
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = calculate_mar(landmarks)

            # Blink detection logic
            if ear < EYE_AR_THRESH:
                blink_frames += 1

                if eyes_closed_time is None:
                    eyes_closed_time = time.time()

                if time.time() - eyes_closed_time >= DROWSINESS_TIME_THRESHOLD:
                    if not drowsy_command_played:
                        thread_play_voice_command("You are feeling drowsy! Stay alert")
                        drowsy_command_played = True
                        
                    else:
                        thread_play_voice_command("You are feeling drowsy! Stay alert")
                        drowsy_img_displayed = True  # Show drowsy image
                        cv2.putText(frame, f'Drowsy state', (30, 400),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                        if time.time() - eyes_closed_time >= EMAIL_ALERT_THRESHOLD and not email_alert_sent:
                            print("Attempting to send email...")
                            send_email()
                            email_alert_sent = True

            else:
                if blink_frames >= EYE_AR_CONSEC_FRAMES:
                    blink_count += 1

                blink_frames = 0
                eyes_closed_time = None
                drowsy_command_played = False
                email_alert_sent = False

            # Yawn detection logic
            if mar > MOUTH_AR_THRESH:
                if not mouth_open and cooldown == 0:
                    mouth_open = True
            elif mar < MOUTH_AR_CLOSE_THRESH:
                if mouth_open:
                    yawn_count += 1
                    mouth_open = False
                    cooldown = COOLDOWN_PERIOD  # Start cooldown period

            # Decrease the cooldown timer
            if cooldown > 0:
                cooldown -= 1

            # Check if yawn count exceeds the threshold
            if yawn_count >= YAWN_LIMIT:
                if not fatigue_command_played:
                    thread_play_voice_command("You look fatigued! Take rest")
                    cv2.putText(frame, f'Fatigue state', (30, 400),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    fatigue_command_played = True
                    fatigue_img_displayed = True  # Show fatigue image

                yawn_count = 0
                yawn_timer_start = time.time()
                fatigue_command_played = False  # Reset the fatigue command flag

            # Timer logic for yawn count reset, independent of the yawn threshold
            if time.time() - yawn_timer_start >= TIMER_DURATION:
                yawn_timer_start = time.time()
                yawn_count = 0
                fatigue_command_played = False

            # Calculate head position
            position = get_head_position(jawline, frame_width, frame_height)

            # Check head position for voice command
            if position != 'center':
                if head_position_start_time is None:
                    head_position_start_time = time.time()

                if time.time() - head_position_start_time >= HEAD_POSITION_TIME_THRESHOLD:
                    thread_play_voice_command("Look at the road")
                    cv2.putText(frame, f'Cognitive Distraction', (30, 400),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    look_at_road_command_played = True
                       
                    look_straight_img_displayed = True  # Show head position image

            else:
                head_position_start_time = None
                look_at_road_command_played = False

            # Calculate pupil centers
            pupil_left = get_pupil_center(left_eye)
            pupil_right = get_pupil_center(right_eye)

    # Display blink and yawn counts
    cv2.putText(frame, f'Blinks: {blink_count}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f'Yawns: {yawn_count}', (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display head position
    cv2.putText(frame, f'Head Position: {position}', (10, 110),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display pupil markers
    cv2.circle(frame, pupil_left, 3, (0, 255, 0), -1)
    cv2.circle(frame, pupil_right, 3, (0, 255, 0), -1)

    # Display date, time, and location
    now = datetime.now()
    date_str = now.strftime('%d-%m-%Y')
    time_str = now.strftime('%H:%M:%S')

    cv2.putText(frame, f'{date_str}', (frame_width - 200, frame_height - 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(frame, f'{time_str}', (frame_width - 200, frame_height - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(frame, f'{location_text}', (10, frame_height - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Overlay images based on flags
    if drowsy_img_displayed:
        # Get frame and overlay image dimensions
        frame_h, frame_w = frame.shape[:2]
        overlay_h, overlay_w = drowsy_img.shape[:2]
    
        # Resize the overlay if it exceeds the frame dimensions
        if overlay_w > frame_w or overlay_h > frame_h:
            scale_factor = min(frame_w / overlay_w, frame_h / overlay_h)
            drowsy_img = cv2.resize(drowsy_img, (int(overlay_w * scale_factor), int(overlay_h * scale_factor)))
    
        # Now, overlay the image after resizing (if needed)
        frame = overlay_image_alpha(frame, drowsy_img, (1100, 10))

    
    if fatigue_img_displayed:
        frame = overlay_image_alpha(frame, fatigue_img, (1100, 10))
    
    # Inside your main processing loop, before overlaying the image
    if look_straight_img_displayed:
        # Get frame and overlay image dimensions
        frame_h, frame_w = frame.shape[:2]
        overlay_h, overlay_w = look_straight_img.shape[:2]
    
        # Resize the overlay if it exceeds the frame dimensions
        if overlay_w > frame_w or overlay_h > frame_h:
            scale_factor = min(frame_w / overlay_w, frame_h / overlay_h)
            look_straight_img = cv2.resize(look_straight_img, (int(overlay_w * scale_factor), int(overlay_h * scale_factor)))
    
        # Now, overlay the image after resizing (if needed)
        frame = overlay_image_alpha(frame, look_straight_img, (1100, 10))


    # Show the frame
    cv2.imshow('Face Landmarks Detection', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

# Posture detection

In [ ]:
import cv2
import mediapipe as mp
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('MediaPipe Pose', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Adaptive cruise control

In [ ]:
import cv2
import torch
import time

# Load the YOLOv5 model pre-trained on the COCO dataset (which includes 'car', 'motorcycle', and 'truck' detection)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Open video file for video capture
video_path = 'C:/Users/DELL/Downloads/lane_vid.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Get the original frame rate of the video
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the known sizes (in meters) for car, motorcycle, and truck
KNOWN_CAR_WIDTH = 1.8  # Average width of a car in meters
KNOWN_BIKE_WIDTH = 0.8  # Average width of a motorcycle in meters
KNOWN_TRUCK_WIDTH = 2.5  # Average width of a truck in meters

FOCAL_LENGTH = 700  # Adjust this value based on the camera's calibration

# Distance threshold in meters
DISTANCE_THRESHOLD = 10  # Adjust this to define how close a vehicle should be for a warning

# Function to estimate distance using pinhole camera model
def estimate_distance(bbox_width, object_width):
    """
    Estimate the distance based on the size of the bounding box using a pinhole camera model.
    """
    # Using the pinhole camera model: distance = (real_width * focal_length) / apparent_width
    distance = (object_width * FOCAL_LENGTH) / bbox_width
    return distance

# Function to simulate braking
def apply_brakes():
    print("Emergency Brake Applied!")  # Simulated braking action

# Real-time object detection and distance estimation
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame to reduce processing time (optional)
    frame = cv2.resize(frame, (640, 360))  # Resize to 640x360 for faster processing

    # Detect objects in the current frame using YOLOv5
    results = model(frame)

    # Process the results to detect 'car', 'motorcycle', and 'truck'
    for result in results.xyxy[0]:  # Iterate through bounding boxes
        x1, y1, x2, y2, conf, class_id = result.tolist()
        label = model.names[int(class_id)]

        if label in ['car', 'motorcycle', 'truck']:  # Check for car, motorcycle, and truck
            # Calculate the width of the bounding box
            bbox_width = x2 - x1
        
            # Set the known width based on the detected object
            if label == 'car':
                object_width = KNOWN_CAR_WIDTH
            elif label == 'motorcycle':
                object_width = KNOWN_BIKE_WIDTH
            elif label == 'truck':
                object_width = KNOWN_TRUCK_WIDTH
        
            # Estimate the distance to the object using the known width
            distance = estimate_distance(bbox_width, object_width)
        
            # Only display bounding box and distance if the object is below the threshold
            if distance < DISTANCE_THRESHOLD:
                # Draw a bounding box around the detected object
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        
                # Display the distance on the frame
                cv2.putText(frame, f"{label.capitalize()} Distance: {distance:.2f}m", (int(x1), int(y1) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        
                # Trigger the braking system if the object is too close
                if distance < 2:
                    apply_brakes()
                    print("Apply Brake")

    # Display the video feed with detections
    cv2.imshow('AEB System', frame)

    # Control the frame rate to match the original video speed
    if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):  # Delay adjusted to video FPS
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()

# Cigar detection

In [ ]:
# good
import torch
import cv2
import pyttsx3
import threading

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Function to speak the warning in a separate thread
def speak_warning():
    engine.say("Don't smoke inside the car")
    engine.runAndWait()

# Load YOLOv5 model (replace the path with your best.pt)
model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/Users/DELL/yolov5/runs/train/exp28/weights/best.pt')

# Set confidence threshold for detections
model.conf = 0.45  # Adjust the confidence threshold if needed

# Open webcam or video feed
cap = cv2.VideoCapture(0)  # 0 for webcam, or provide path to video file

# Create a flag to prevent multiple threads from speaking the warning simultaneously
warning_thread = None

frame_count = 0
inference_interval = 3  # Run YOLO inference every 3 frames

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1

    # Run inference only on every nth frame
    if frame_count % inference_interval == 0:
        results = model(frame)
        labels = results.xyxyn[0][:, -1].cpu().numpy()  # Get the labels of detected objects
        names = results.names  # Get names of the labels (e.g., 'cigar')
        
        # Extract bounding boxes and confidence scores
        boxes = results.xyxy[0].cpu().numpy()  # Get the bounding boxes (x1, y1, x2, y2, confidence, class)

        # Loop through the detections
        for box in boxes:
            x1, y1, x2, y2, conf, cls = box  # Unpack the bounding box
            label = names[int(cls)]  # Get the label name

            # Draw the bounding box on the frame if 'cigar' is detected
            if label == 'cigar':
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Draw the bounding box
                cv2.putText(frame, f'{label} {conf:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Check for cigar detection to trigger voice command
        if any(names[int(label)] == 'cigar' for label in labels):
            if warning_thread is None or not warning_thread.is_alive():
                warning_thread = threading.Thread(target=speak_warning)
                warning_thread.start()

    # Display the output
    cv2.imshow('Smoke Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
